In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [48]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

url = 'https://www.google.com/finance/?hl=ja'
driver.get(url)

time.sleep(1)


In [49]:
stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NVDA")
stock_code_input.send_keys(Keys.RETURN)

time.sleep(3)

In [50]:
driver.find_element(By.XPATH, r'//*[@id="6monthTab"]').click()

logs = driver.get_log('performance')

In [51]:
current_url = driver.current_url
new_url = f"{current_url}&window=6M"
driver.get(new_url)
logs = driver.get_log('performance')

In [ ]:
print(logs)

In [52]:
batch_request_urls = []
headers = {}
post_data = None

stock_code = "NVDA"

for entry in logs:
    log = json.loads(entry['message'])['message']
    try:
        if 'Network.requestWillBeSent' in log['method']:
            request_url = log['params']['request']['url']
            if f'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2F{stock_code}' in request_url:
                batch_request_urls.append(request_url)
                headers = log['params']['request']['headers']
                if 'postData' in log['params']['request']:
                    post_data = log['params']['request']['postData']
    except KeyError:
        continue


In [53]:
# RequestsでGETリクエストを送信し、特定の文字列が含まれるレスポンスを探す
target_string = fr'[["wrb.fr","AiCwsd","[[[[\"{stock_code}\",'

for url in batch_request_urls:
    if post_data:
        response = requests.post(url, headers=headers, data=post_data)
    else:
        response = requests.get(url, headers=headers)

    print(f"Checking URL: {url}")

    print(response.text)
    if target_string in response.text and stock_code in response.text:
        print("Found target string in the response!")
        print("Status Code:", response.status_code)
        print("Response Text:", response.text)
        # try:
        #     data = response.json()
        #     print(data)
        # except json.JSONDecodeError:
        #     print("レスポンスのJSONデコードに失敗しました")
        # break
else:
    print("対象の文字列を含むレスポンスが見つかりませんでした")

対象の文字列を含むレスポンスが見つかりませんでした


In [9]:
text = r''')
]
}'

106339
[["wrb.fr","AiCwsd","[[[[\".INX\",\"INDEXSP\"],\"/m/016yss\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[4927.93,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[4924.97,-2.9600000000000364,-6.006578827215558E-4,2,2,5]],[[2024,1,31,16,null,null,null,[-14400]],[4845.65,-82.28000000000065,-0.016696665739976146,2,2,3]],[[2024,2,1,16,null,null,null,[-14400]],[4906.19,-21.74000000000069,-0.004411588638637458,2,2,4]],[[2024,2,2,16,null,null,null,[-14400]],[4958.61,30.67999999999938,0.006225737784424572,2,2,4]],[[2024,2,5,16,null,null,null,[-14400]],[4942.81,14.88000000000011,0.003019523410438076,2,2,4]],[[2024,2,6,16,null,null,null,[-14400]],[4954.23,26.299999999999272,0.005336926457964961,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[4995.06,67.13000000000011,0.013622352590235679,2,2,3]],[[2024,2,8,16,null,null,null,[-14400]],[4997.91,69.97999999999956,0.014200688727315437,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[5026.61,98.67999999999938,0.020024635090189872,2,2,3]],[[2024,2,12,16,null,null,null,[-14400]],[5021.84,93.90999999999985,0.01905668302918261,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[4953.17,25.23999999999978,0.005121825999963429,2,2,4]],[[2024,2,14,16,null,null,null,[-14400]],[5000.62,72.6899999999996,0.014750615369942268,2,2,3]],[[2024,2,15,16,null,null,null,[-14400]],[5029.73,101.79999999999927,0.02065776096657202,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[5005.57,77.63999999999942,0.01575509392381779,2,2,3]],[[2024,2,20,16,null,null,null,[-14400]],[4975.51,47.57999999999993,0.009655169614828117,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[4981.8,53.86999999999989,0.0109315676156114,2,2,3]],[[2024,2,22,16,null,null,null,[-14400]],[5087.03,159.09999999999945,0.03228536119628311,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[5088.8,160.8699999999999,0.03264453837615386,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[5069.53,141.59999999999945,0.02873417438965234,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[5078.18,150.25,0.030489475296929948,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[5069.76,141.82999999999993,0.02878084713053958,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[5096.27,168.34000000000015,0.034160387830184304,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[5137.08,209.14999999999964,0.04244175546324717,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[5130.95,203.01999999999953,0.04119782545612448,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[5078.65,150.71999999999935,0.030584850028307898,2,2,3]],[[2024,3,6,16,null,null,null,[-14400]],[5104.76,176.82999999999993,0.03588322074380113,2,2,3]],[[2024,3,7,16,null,null,null,[-14400]],[5157.36,229.42999999999938,0.04655707365973124,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[5123.69,195.7599999999993,0.03972459024377361,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[5117.94,190.0099999999993,0.03855777172159493,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[5175.27,247.34000000000015,0.05019145970011752,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[5165.31,237.3800000000001,0.048170327094743656,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[5150.48,222.54999999999927,0.045160949932324376,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[5117.09,189.15999999999985,0.038385285505272974,2,2,3]],[[2024,3,18,16,null,null,null,[-14400]],[5149.42,221.48999999999978,0.04494584947432284,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[5178.51,250.57999999999993,0.050848936571745114,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[5224.62,296.6899999999996,0.060205806494816196,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[5241.53,313.59999999999945,0.0636372675748234,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[5234.18,306.25,0.06214576911603858,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[5218.19,290.2599999999993,0.058900998999579804,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[5203.58,275.64999999999964,0.05593626532844412,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[5248.49,320.5599999999995,0.06504962529906055,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[5254.35,326.4200000000001,0.06623876556688103,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[5243.77,315.84000000000015,0.06409181948607227,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[5205.81,277.8800000000001,0.056388787990089166,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[5211.49,283.5599999999995,0.057541401765041196,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[5147.21,219.27999999999975,0.04449738531188546,2,2,3]],[[2024,4,5,16,null,null,null,[-14400]],[5204.34,276.40999999999985,0.056090488298332124,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[5202.39,274.46000000000004,0.055694784625593304,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[5209.91,281.97999999999956,0.05722078032764255,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[5160.64,232.71000000000004,0.04722266752977417,2,2,3]],[[2024,4,11,16,null,null,null,[-14400]],[5199.06,271.1300000000001,0.055019044507531575,2,2,3]],[[2024,4,12,16,null,null,null,[-14400]],[5123.41,195.47999999999956,0.03966777125486757,2,2,3]],[[2024,4,15,16,null,null,null,[-14400]],[5061.82,133.88999999999942,0.027169622945131,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[5051.41,123.47999999999956,0.025057174107586667,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[5022.21,94.27999999999975,0.01913176526452278,2,2,3]],[[2024,4,18,16,null,null,null,[-14400]],[5011.12,83.1899999999996,0.016881327453920732,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[4967.23,39.29999999999927,0.00797495094289068,2,2,4]],[[2024,4,22,16,null,null,null,[-14400]],[5010.6,82.67000000000007,0.0167758064745238,2,2,3]],[[2024,4,23,16,null,null,null,[-14400]],[5070.55,142.6199999999999,0.028941157849238907,2,2,3]],[[2024,4,24,16,null,null,null,[-14400]],[5071.63,143.69999999999982,0.029160316806448105,2,2,3]],[[2024,4,25,16,null,null,null,[-14400]],[5048.42,120.48999999999978,0.024450428476053793,2,2,3]],[[2024,4,26,16,null,null,null,[-14400]],[5099.96,172.02999999999975,0.03490918093398237,2,2,3]],[[2024,4,29,16,null,null,null,[-14400]],[5116.17,188.23999999999978,0.03819859454172437,2,2,3]],[[2024,4,30,16,null,null,null,[-14400]],[5035.69,107.75999999999931,0.021867193730430282,2,2,3]],[[2024,5,1,16,null,null,null,[-14400]],[5018.39,90.46000000000004,0.018356591915875433,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[5064.2,136.26999999999953,0.027652584350832808,2,2,3]],[[2024,5,3,16,null,null,null,[-14400]],[5127.79,199.85999999999967,0.04055658258132718,2,2,3]],[[2024,5,6,16,null,null,null,[-14400]],[5180.74,252.8099999999995,0.05130145923338998,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[5187.7,259.7699999999995,0.05271381695762714,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[5187.67,259.7399999999998,0.05270772920881583,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[5214.08,286.14999999999964,0.05806697741242258,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[5222.68,294.75,0.05981213207168121,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[5221.42,293.4899999999998,0.05955644662160375,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[5246.68,318.75,0.06468233112077484,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[5308.15,380.21999999999935,0.07715612843526579,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[5297.1,369.1700000000001,0.07491380762307907,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[5303.27,375.34000000000015,0.07616585462861691,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[5308.13,380.1999999999998,0.0771520699360583,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[5321.41,393.47999999999956,0.07984691340989006,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[5307.01,379.0799999999999,0.07692479398043396,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[5267.84,339.90999999999985,0.06897622328239238,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[5304.72,376.78999999999996,0.07646009582116628,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[5306.04,378.1099999999997,0.07672795676886637,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[5266.95,339.0199999999995,0.06879562006765508,2,2,3]],[[2024,5,30,16,null,null,null,[-14400]],[5235.48,307.5499999999993,0.062409571564531,2,2,3]],[[2024,5,31,16,null,null,null,[-14400]],[5277.51,349.5799999999999,0.07093850764925637,2,2,3]],[[2024,6,3,16,null,null,null,[-14400]],[5283.4,355.46999999999935,0.07213373566588797,2,2,3]],[[2024,6,4,16,null,null,null,[-14400]],[5291.34,363.40999999999985,0.07374495985129656,2,2,3]],[[2024,6,5,16,null,null,null,[-14400]],[5354.03,426.09999999999945,0.08646632561744981,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[5352.96,425.02999999999975,0.08624919590984444,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[5346.99,419.0599999999995,0.08503773389638235,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[5360.79,432.8599999999997,0.08783809834961123,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[5375.32,447.3899999999994,0.09078659802391661,2,2,3]],[[2024,6,12,16,null,null,null,[-14400]],[5421.03,493.09999999999945,0.10006229796283621,2,2,2]],[[2024,6,13,16,null,null,null,[-14400]],[5433.74,505.8099999999995,0.10264147420925206,2,2,2]],[[2024,6,14,16,null,null,null,[-14400]],[5431.6,503.6700000000001,0.10220721479404132,2,2,2]],[[2024,6,17,16,null,null,null,[-14400]],[5473.23,545.2999999999993,0.11065498089461483,2,2,2]],[[2024,6,18,16,null,null,null,[-14400]],[5487.03,559.0999999999995,0.1134553453478437,2,2,2]],[[2024,6,20,16,null,null,null,[-14400]],[5473.17,545.2399999999998,0.1106428053969922,2,2,2]],[[2024,6,21,16,null,null,null,[-14400]],[5464.62,536.6899999999996,0.10890779698575255,2,2,2]],[[2024,6,24,16,null,null,null,[-14400]],[5447.87,519.9399999999996,0.10550880389940595,2,2,2]],[[2024,6,25,16,null,null,null,[-14400]],[5469.3,541.3699999999999,0.10985748580032587,2,2,2]],[[2024,6,26,16,null,null,null,[-14400]],[5477.9,549.9699999999993,0.1116026404595843,2,2,2]],[[2024,6,27,16,null,null,null,[-14400]],[5482.87,554.9399999999996,0.1126111775126675,2,2,2]],[[2024,6,28,16,null,null,null,[-14400]],[5460.48,532.5499999999993,0.10806768764978383,2,2,2]],[[2024,7,1,16,null,null,null,[-14400]],[5475.09,547.1599999999999,0.1110324213209197,2,2,2]],[[2024,7,2,16,null,null,null,[-14400]],[5509.01,581.0799999999999,0.11791563597697205,2,2,2]],[[2024,7,3,13,null,null,null,[-14400]],[5537.02,609.0900000000001,0.12359956411718513,2,2,2]],[[2024,7,5,16,null,null,null,[-14400]],[5567.19,639.2599999999993,0.12972181017181642,2,2,2]],[[2024,7,8,16,null,null,null,[-14400]],[5572.85,644.9200000000001,0.13087036544756114,2,2,2]],[[2024,7,9,16,null,null,null,[-14400]],[5576.98,649.0499999999993,0.13170844553392586,2,2,2]],[[2024,7,10,16,null,null,null,[-14400]],[5633.91,705.9799999999996,0.1432609635282968,2,2,2]],[[2024,7,11,16,null,null,null,[-14400]],[5584.54,656.6099999999997,0.13324255823439043,2,2,2]],[[2024,7,12,16,null,null,null,[-14400]],[5615.35,687.4200000000001,0.13949467626366446,2,2,2]],[[2024,7,15,16,null,null,null,[-14400]],[5631.22,703.29,0.14271509538487762,2,2,2]],[[2024,7,16,16,null,null,null,[-14400]],[5667.2,739.2699999999995,0.1500163354593104,2,2,2]],[[2024,7,17,16,null,null,null,[-14400]],[5588.27,660.3400000000001,0.13399946833660384,2,2,2]],[[2024,7,18,16,null,null,null,[-14400]],[5544.59,616.6599999999999,0.12513570606725335,2,2,2]],[[2024,7,19,16,null,null,null,[-14400]],[5505,577.0699999999997,0.11710190688585262,2,2,2]],[[2024,7,22,16,null,null,null,[-14400]],[5564.41,636.4799999999996,0.12915767878196313,2,2,2]],[[2024,7,23,16,null,null,null,[-14400]],[5555.74,627.8099999999995,0.12739831937547802,2,2,2]],[[2024,7,24,16,null,null,null,[-14400]],[5427.13,499.1999999999998,0.10130014022114758,2,2,2]],[[2024,7,25,16,null,null,null,[-14400]],[5399.22,471.28999999999996,0.09563650457697247,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[5459.1,531.1700000000001,0.10778765120446111,2,2,2]],[[2024,7,26,17,9,24,null,[-14400]],[5459.1,531.1700000000001,0.10778765120446111,2,2,2]]]]],null,-14400,5399.22,\"S\\u0026P 500\",86400,1],[[\".DJI\",\"INDEXDJX\"],\"/m/0cqyw\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[38333.45,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[38467.31,133.86000000000058,0.003491989372206274,2,2,4]],[[2024,1,31,16,null,null,null,[-14400]],[38150.3,-183.14999999999418,-0.004777811545790796,2,2,4]],[[2024,2,1,16,null,null,null,[-14400]],[38519.84,186.38999999999942,0.0048623330276820744,2,2,4]],[[2024,2,2,16,null,null,null,[-14400]],[38654.42,320.97000000000116,0.008373104951419744,2,2,4]],[[2024,2,5,16,null,null,null,[-14400]],[38380.12,46.67000000000553,0.0012174745555123666,2,2,4]],[[2024,2,6,16,null,null,null,[-14400]],[38521.36,187.9100000000035,0.004901985080915063,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[38677.36,343.9100000000035,0.008971537912710792,2,2,4]],[[2024,2,8,16,null,null,null,[-14400]],[38726.33,392.88000000000466,0.010249012285614905,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[38671.69,338.24000000000524,0.008823625319401339,2,2,4]],[[2024,2,12,16,null,null,null,[-14400]],[38797.38,463.9300000000003,0.01210248490548073,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[38272.75,-60.69999999999709,-0.0015834734416024932,2,2,4]],[[2024,2,14,16,null,null,null,[-14400]],[38424.27,90.81999999999971,0.00236921018066466,2,2,4]],[[2024,2,15,16,null,null,null,[-14400]],[38773.12,439.67000000000553,0.011469617266382378,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[38627.99,294.5400000000009,0.007683628788955883,2,2,4]],[[2024,2,20,16,null,null,null,[-14400]],[38563.8,230.35000000000582,0.006009112146180577,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[38612.24,278.7900000000009,0.0072727604742072755,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[39069.11,735.6600000000035,0.019191072027172185,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[39131.53,798.0800000000017,0.02081941489743297,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[39069.23,735.7800000000061,0.019194202452427478,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[38972.41,638.9600000000064,0.016668471009001446,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[38949.02,615.5699999999997,0.01605829895300318,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[38996.39,662.9400000000023,0.017294034322504297,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[39087.38,753.9300000000003,0.019667679272280483,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[38989.83,656.3800000000047,0.017122904408551923,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[38585.19,251.74000000000524,0.0065671104479248605,2,2,4]],[[2024,3,6,16,null,null,null,[-14400]],[38661.05,327.6000000000058,0.008546060946771183,2,2,4]],[[2024,3,7,16,null,null,null,[-14400]],[38791.35,457.90000000000146,0.01194518103640558,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[38722.69,389.24000000000524,0.01015405605287302,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[38769.66,436.2100000000064,0.011379356671523342,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[39005.49,672.0400000000009,0.01753142490435901,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[39043.32,709.8700000000026,0.018518291466069522,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[38905.66,572.2100000000064,0.014927171960781157,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[38714.77,381.3199999999997,0.009947447986027862,2,2,4]],[[2024,3,18,16,null,null,null,[-14400]],[38790.43,456.9800000000032,0.011921181109448882,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[39110.76,777.310000000005,0.020277590459507427,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[39512.13,1178.6800000000003,0.030748080331929436,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[39781.37,1447.9200000000055,0.03777171112957497,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[39475.9,1142.4500000000044,0.029802952773622108,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[39313.64,980.1900000000023,0.02557009609101196,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[39282.33,948.8800000000047,0.024753315968168915,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[39760.08,1426.6300000000047,0.03721632151554334,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[39807.37,1473.9200000000055,0.03844996993487426,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[39566.85,1233.4000000000015,0.0321755542483132,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[39170.24,836.7900000000009,0.021829237911015077,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[39127.14,793.6900000000023,0.02070489350684591,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[38596.98,263.5300000000061,0.006874674729250984,2,2,4]],[[2024,4,5,16,null,null,null,[-14400]],[38904.04,570.5900000000038,0.014884911219835519,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[38892.8,559.3500000000058,0.014591694720929263,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[38883.67,550.2200000000012,0.014353521532760583,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[38461.51,128.06000000000495,0.003340685484870393,2,2,4]],[[2024,4,11,16,null,null,null,[-14400]],[38459.08,125.63000000000466,0.0032772943734520285,2,2,4]],[[2024,4,12,16,null,null,null,[-14400]],[37983.24,-350.2099999999991,-0.009135885238610121,2,2,4]],[[2024,4,15,16,null,null,null,[-14400]],[37735.11,-598.3399999999965,-0.015608822060106683,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[37798.97,-534.4799999999959,-0.01394291408678311,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[37753.31,-580.1399999999994,-0.015134040896397258,2,2,3]],[[2024,4,18,16,null,null,null,[-14400]],[37775.38,-558.0699999999997,-0.014558303518206677,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[37986.4,-347.04999999999563,-0.00905345070688904,2,2,4]],[[2024,4,22,16,null,null,null,[-14400]],[38239.98,-93.46999999999389,-0.0024383404050507818,2,2,4]],[[2024,4,23,16,null,null,null,[-14400]],[38503.69,170.24000000000524,0.004441029962082861,2,2,4]],[[2024,4,24,16,null,null,null,[-14400]],[38460.92,127.47000000000116,0.0033252942273654254,2,2,4]],[[2024,4,25,16,null,null,null,[-14400]],[38085.8,-247.64999999999418,-0.006460415120475569,2,2,4]],[[2024,4,26,16,null,null,null,[-14400]],[38239.66,-93.7899999999936,-0.002446688205731381,2,2,4]],[[2024,4,29,16,null,null,null,[-14400]],[38386.09,52.63999999999942,0.0013732132119597747,2,2,4]],[[2024,4,30,16,null,null,null,[-14400]],[37815.92,-517.5299999999988,-0.013500741519482303,2,2,3]],[[2024,5,1,16,null,null,null,[-14400]],[37903.29,-430.1599999999962,-0.011221531064905357,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[38225.66,-107.7899999999936,-0.002811904485507921,2,2,4]],[[2024,5,3,16,null,null,null,[-14400]],[38675.68,342.2300000000032,0.0089277119591376,2,2,4]],[[2024,5,6,16,null,null,null,[-14400]],[38852.27,518.8199999999997,0.01353439359097602,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[38884.26,550.810000000005,0.01436891279026555,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[39056.39,722.9400000000023,0.01885924695011804,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[39387.76,1054.310000000005,0.027503655423657538,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[39512.84,1179.3899999999994,0.030766602014689507,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[39431.51,1098.060000000005,0.028644956297959226,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[39558.11,1224.6600000000035,0.031947554942224184,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[39908,1574.550000000003,0.04107509238015371,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[39869.38,1535.9300000000003,0.040067617185512924,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[40003.59,1670.1399999999994,0.04356873696471358,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[39806.77,1473.3199999999997,0.03843431780859797,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[39872.99,1539.5400000000009,0.04016179081194103,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[39671.04,1337.5900000000038,0.03489354597616452,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[39065.26,731.810000000005,0.01909063755023367,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[39069.59,736.1399999999994,0.019203593728192987,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[38852.86,519.4100000000035,0.013549784848480987,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[38441.54,108.09000000000378,0.002819730548646255,2,2,4]],[[2024,5,30,16,null,null,null,[-14400]],[38111.48,-221.9699999999939,-0.00579050411585688,2,2,4]],[[2024,5,31,16,null,null,null,[-14400]],[38686.32,352.8700000000026,0.009205276331767756,2,2,4]],[[2024,6,3,16,null,null,null,[-14400]],[38571.03,237.58000000000175,0.006197720267807926,2,2,4]],[[2024,6,4,16,null,null,null,[-14400]],[38711.29,377.8400000000038,0.00985666565362637,2,2,4]],[[2024,6,5,16,null,null,null,[-14400]],[38807.33,473.88000000000466,0.012362049332893458,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[38886.17,552.7200000000012,0.014418738725577824,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[38798.99,465.5400000000009,0.012144484777655049,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[38868.04,534.5900000000038,0.013945783643267273,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[38747.42,413.97000000000116,0.010799184524221045,2,2,3]],[[2024,6,12,16,null,null,null,[-14400]],[38712.21,378.76000000000204,0.00988066558058307,2,2,4]],[[2024,6,13,16,null,null,null,[-14400]],[38647.1,313.65000000000146,0.008182149010850875,2,2,4]],[[2024,6,14,16,null,null,null,[-14400]],[38589.16,255.7100000000064,0.006670675350118668,2,2,4]],[[2024,6,17,16,null,null,null,[-14400]],[38778.1,444.65000000000146,0.011599529914474212,2,2,3]],[[2024,6,18,16,null,null,null,[-14400]],[38834.86,501.4100000000035,0.013080221060196866,2,2,3]],[[2024,6,20,16,null,null,null,[-14400]],[39134.76,801.310000000005,0.020903675510552926,2,2,3]],[[2024,6,21,16,null,null,null,[-14400]],[39150.33,816.8800000000047,0.021309848187418684,2,2,3]],[[2024,6,24,16,null,null,null,[-14400]],[39411.21,1077.760000000002,0.028115392692283164,2,2,3]],[[2024,6,25,16,null,null,null,[-14400]],[39112.16,778.7100000000064,0.02031411208748512,2,2,3]],[[2024,6,26,16,null,null,null,[-14400]],[39127.8,794.3500000000058,0.020722110845749753,2,2,3]],[[2024,6,27,16,null,null,null,[-14400]],[39164.06,830.6100000000006,0.021668021010370856,2,2,3]],[[2024,6,28,16,null,null,null,[-14400]],[39118.86,785.4100000000035,0.020488894164235244,2,2,3]],[[2024,7,1,16,null,null,null,[-14400]],[39169.52,836.0699999999997,0.021810455359483682,2,2,3]],[[2024,7,2,16,null,null,null,[-14400]],[39331.85,998.4000000000015,0.026045138123492708,2,2,3]],[[2024,7,3,13,null,null,null,[-14400]],[39308,974.5500000000029,0.025422966104016283,2,2,3]],[[2024,7,5,16,null,null,null,[-14400]],[39375.87,1042.4200000000055,0.027193482454618764,2,2,3]],[[2024,7,8,16,null,null,null,[-14400]],[39344.79,1011.3400000000038,0.0263827023135148,2,2,3]],[[2024,7,9,16,null,null,null,[-14400]],[39291.97,958.5200000000041,0.025004793463672174,2,2,3]],[[2024,7,10,16,null,null,null,[-14400]],[39721.36,1387.9100000000035,0.036206237633189906,2,2,3]],[[2024,7,11,16,null,null,null,[-14400]],[39753.75,1420.300000000003,0.03705119158333004,2,2,3]],[[2024,7,12,16,null,null,null,[-14400]],[40000.9,1667.4500000000044,0.04349856326524235,2,2,3]],[[2024,7,15,16,null,null,null,[-14400]],[40211.72,1878.270000000004,0.0489981987011345,2,2,3]],[[2024,7,16,16,null,null,null,[-14400]],[40954.48,2621.030000000006,0.06837448755590761,2,2,3]],[[2024,7,17,16,null,null,null,[-14400]],[41198.08,2864.6300000000047,0.07472925082401936,2,2,3]],[[2024,7,18,16,null,null,null,[-14400]],[40665.02,2331.5699999999997,0.0608233801027562,2,2,3]],[[2024,7,19,16,null,null,null,[-14400]],[40287.53,1954.0800000000017,0.0509758448561244,2,2,3]],[[2024,7,22,16,null,null,null,[-14400]],[40415.44,2081.9900000000052,0.05431261730942572,2,2,3]],[[2024,7,23,16,null,null,null,[-14400]],[40358.09,2024.6399999999994,0.0528165349061981,2,2,3]],[[2024,7,24,16,null,null,null,[-14400]],[39853.87,1520.4200000000055,0.03966300972127491,2,2,3]],[[2024,7,25,16,null,null,null,[-14400]],[39935.07,1601.6200000000026,0.041781264143978765,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[40589.34,2255.8899999999994,0.058849125241792734,2,2,3]],[[2024,7,26,17,9,23,null,[-14400]],[40589.34,2255.8899999999994,0.058849125241792734,2,2,3]]]]],null,-14400,39935.07,\"ダウ平均株価\",86400,1],[[\".IXIC\",\"INDEXNASDAQ\"],\"/m/02853rl\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[15628.04,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[15509.9,-118.14000000000124,-0.007559489225776312,2,2,4]],[[2024,1,31,16,null,null,null,[-14400]],[15164.01,-464.03000000000065,-0.02969214309663916,2,2,3]],[[2024,2,1,16,null,null,null,[-14400]],[15361.64,-266.40000000000146,-0.01704628347508718,2,2,3]],[[2024,2,2,16,null,null,null,[-14400]],[15628.95,0.9099999999998545,5.822867102975514E-5,2,2,6]],[[2024,2,5,16,null,null,null,[-14400]],[15597.68,-30.360000000000582,-0.0019426620356743764,2,2,4]],[[2024,2,6,16,null,null,null,[-14400]],[15609,-19.040000000000873,-0.0012183229630843581,2,2,4]],[[2024,2,7,16,null,null,null,[-14400]],[15756.64,128.59999999999854,0.00822879900486552,2,2,4]],[[2024,2,8,16,null,null,null,[-14400]],[15793.72,165.67999999999847,0.010601457380451961,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[15990.66,362.619999999999,0.023203165592102334,2,2,3]],[[2024,2,12,16,null,null,null,[-14400]],[15942.55,314.5099999999984,0.02012472453359464,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[15655.6,27.55999999999949,0.001763496894044262,2,2,4]],[[2024,2,14,16,null,null,null,[-14400]],[15859.15,231.10999999999876,0.014788162815042626,2,2,3]],[[2024,2,15,16,null,null,null,[-14400]],[15906.17,278.1299999999992,0.017796857443415758,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[15775.65,147.60999999999876,0.009445202341432372,2,2,4]],[[2024,2,20,16,null,null,null,[-14400]],[15630.78,2.7399999999997817,1.7532588859510095E-4,2,2,5]],[[2024,2,21,16,null,null,null,[-14400]],[15580.87,-47.17000000000007,-0.003018292760960432,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[16041.62,413.5799999999999,0.026463971169769205,2,2,3]],[[2024,2,23,16,null,null,null,[-14400]],[15996.82,368.77999999999884,0.023597328903688423,2,2,3]],[[2024,2,26,16,null,null,null,[-14400]],[15976.25,348.2099999999991,0.022281104988213438,2,2,3]],[[2024,2,27,16,null,null,null,[-14400]],[16035.3,407.2599999999984,0.026059569850089864,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[15947.74,319.6999999999989,0.020456819921116077,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[16091.92,463.8799999999992,0.029682544964051743,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[16274.94,646.8999999999996,0.04139354647159846,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[16207.51,579.4699999999993,0.03707886593584348,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[15939.59,311.5499999999993,0.019935321383871506,2,2,3]],[[2024,3,6,16,null,null,null,[-14400]],[16031.54,403.5,0.025818976659901047,2,2,3]],[[2024,3,7,16,null,null,null,[-14400]],[16273.38,645.3399999999983,0.04129372589269021,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[16085.11,457.0699999999997,0.029246789744587273,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[16019.27,391.22999999999956,0.025033849414257934,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[16265.64,637.5999999999985,0.04079846225118432,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[16177.77,549.7299999999996,0.03517587618153009,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[16128.53,500.4899999999998,0.03202512919086461,2,2,3]],[[2024,3,15,16,null,null,null,[-14400]],[15973.17,345.1299999999992,0.02208402333242039,2,2,3]],[[2024,3,18,16,null,null,null,[-14400]],[16103.45,475.40999999999985,0.030420321422264075,2,2,3]],[[2024,3,19,16,null,null,null,[-14400]],[16166.79,538.75,0.03447329287613802,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[16369.41,741.369999999999,0.047438450375094955,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[16401.84,773.7999999999993,0.04951356664047438,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[16428.82,800.7799999999988,0.051239950755180994,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[16384.47,756.4300000000003,0.04840210288686235,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[16315.7,687.6599999999999,0.044001679033327266,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[16399.52,771.4799999999996,0.04936511552312379,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[16379.46,751.4199999999983,0.048081525258445605,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[16396.83,768.7900000000009,0.04919298901205787,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[16240.45,612.4099999999999,0.03918661585201982,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[16277.46,649.4199999999983,0.041554795099065414,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[16049.08,421.03999999999905,0.026941318297112053,2,2,3]],[[2024,4,5,16,null,null,null,[-14400]],[16248.52,620.4799999999996,0.03970299538521782,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[16253.96,625.9199999999983,0.040051087660384684,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[16306.64,678.5999999999985,0.043421951825052825,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[16170.36,542.3199999999997,0.0347017284317163,2,2,3]],[[2024,4,11,16,null,null,null,[-14400]],[16442.2,814.1599999999999,0.052096104181970346,2,2,3]],[[2024,4,12,16,null,null,null,[-14400]],[16175.09,547.0499999999993,0.035004389545969884,2,2,3]],[[2024,4,15,16,null,null,null,[-14400]],[15885.02,256.97999999999956,0.016443520748603124,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[15865.25,237.20999999999913,0.015178486873593817,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[15683.37,55.32999999999993,0.003540431173710838,2,2,4]],[[2024,4,18,16,null,null,null,[-14400]],[15601.5,-26.540000000000873,-0.0016982295924505486,2,2,4]],[[2024,4,19,16,null,null,null,[-14400]],[15282.01,-346.03000000000065,-0.022141612127944427,2,2,3]],[[2024,4,22,16,null,null,null,[-14400]],[15451.31,-176.73000000000138,-0.011308519814385002,2,2,3]],[[2024,4,23,16,null,null,null,[-14400]],[15696.64,68.59999999999854,0.004389545969935996,2,2,4]],[[2024,4,24,16,null,null,null,[-14400]],[15712.75,84.70999999999913,0.00542038540981461,2,2,4]],[[2024,4,25,16,null,null,null,[-14400]],[15611.76,-16.280000000000655,-0.001041717323477586,2,2,4]],[[2024,4,26,16,null,null,null,[-14400]],[15927.9,299.85999999999876,0.019187306917566038,2,2,3]],[[2024,4,29,16,null,null,null,[-14400]],[15983.08,355.03999999999905,0.022718139958689574,2,2,3]],[[2024,4,30,16,null,null,null,[-14400]],[15657.82,29.779999999998836,0.0019055492563366126,2,2,4]],[[2024,5,1,16,null,null,null,[-14400]],[15605.48,-22.56000000000131,-0.0014435591411335847,2,2,4]],[[2024,5,2,16,null,null,null,[-14400]],[15840.96,212.91999999999825,0.013624229269953126,2,2,3]],[[2024,5,3,16,null,null,null,[-14400]],[16156.33,528.289999999999,0.03380398309704857,2,2,3]],[[2024,5,6,16,null,null,null,[-14400]],[16349.25,721.2099999999991,0.04614846135535864,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[16332.56,704.5199999999986,0.04508050913614238,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[16302.76,674.7199999999993,0.0431736801287941,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[16346.27,718.2299999999996,0.045957778454623836,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[16340.87,712.8299999999999,0.0456122456814802,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[16388.24,760.2000000000007,0.04864333595255711,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[16511.18,883.1399999999994,0.05650996542112763,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[16742.39,1114.3499999999985,0.0713045269912285,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[16698.32,1070.2799999999988,0.06848459563707278,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[16685.97,1057.9300000000003,0.06769434938738321,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[16794.87,1166.829999999998,0.07466259364578015,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[16832.62,1204.579999999998,0.07707812368025664,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[16801.54,1173.5,0.07508939060816328,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[16736.03,1107.989999999998,0.07089756616952592,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[16920.79,1292.75,0.0827199060150857,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[17019.88,1391.8400000000001,0.08906043240227182,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[16920.58,1292.5400000000009,0.0827064686294635,2,2,3]],[[2024,5,30,16,null,null,null,[-14400]],[16737.08,1109.0400000000009,0.07096475309763738,2,2,3]],[[2024,5,31,16,null,null,null,[-14400]],[16735.02,1106.9799999999996,0.07083293874343805,2,2,3]],[[2024,6,3,16,null,null,null,[-14400]],[16828.67,1200.6299999999974,0.07682537285545707,2,2,3]],[[2024,6,4,16,null,null,null,[-14400]],[16857.05,1229.0099999999984,0.0786413395409788,2,2,3]],[[2024,6,5,16,null,null,null,[-14400]],[17187.91,1559.869999999999,0.09981226052659188,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[17173.12,1545.079999999998,0.09886588465348169,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[17133.13,1505.0900000000001,0.09630702250570129,2,2,3]],[[2024,6,10,16,null,null,null,[-14400]],[17192.53,1564.489999999998,0.10010788301028138,2,2,2]],[[2024,6,11,16,null,null,null,[-14400]],[17343.55,1715.5099999999984,0.10977128289919902,2,2,2]],[[2024,6,12,16,null,null,null,[-14400]],[17608.44,1980.3999999999978,0.12672094517290702,2,2,2]],[[2024,6,13,16,null,null,null,[-14400]],[17667.56,2039.5200000000004,0.1305038891633244,2,2,2]],[[2024,6,14,16,null,null,null,[-14400]],[17688.88,2060.84,0.13186810374173602,2,2,2]],[[2024,6,17,16,null,null,null,[-14400]],[17857.02,2228.9799999999996,0.14262697049662015,2,2,2]],[[2024,6,18,16,null,null,null,[-14400]],[17862.23,2234.1899999999987,0.14296034563515314,2,2,2]],[[2024,6,20,16,null,null,null,[-14400]],[17721.59,2093.5499999999993,0.13396113652127836,2,2,2]],[[2024,6,21,16,null,null,null,[-14400]],[17689.36,2061.3199999999997,0.13189881776601542,2,2,2]],[[2024,6,24,16,null,null,null,[-14400]],[17496.82,1868.7799999999988,0.11957865477692652,2,2,2]],[[2024,6,25,16,null,null,null,[-14400]],[17717.65,2089.6100000000006,0.13370902557198475,2,2,2]],[[2024,6,26,16,null,null,null,[-14400]],[17805.16,2177.119999999999,0.13930857612342934,2,2,2]],[[2024,6,27,16,null,null,null,[-14400]],[17858.68,2230.6399999999994,0.14273318983058653,2,2,2]],[[2024,6,28,16,null,null,null,[-14400]],[17732.6,2104.5599999999977,0.13466563945318782,2,2,2]],[[2024,7,1,16,null,null,null,[-14400]],[17879.3,2251.2599999999984,0.14405261312359055,2,2,2]],[[2024,7,2,16,null,null,null,[-14400]],[18028.76,2400.7199999999975,0.15361619243359995,2,2,2]],[[2024,7,3,13,5,null,null,[-14400]],[18188.3,2560.2599999999984,0.1638247662534776,2,2,2]],[[2024,7,5,16,null,null,null,[-14400]],[18352.76,2724.7199999999975,0.17434815882221938,2,2,2]],[[2024,7,8,16,null,null,null,[-14400]],[18403.74,2775.7000000000007,0.17761024415089804,2,2,2]],[[2024,7,9,16,null,null,null,[-14400]],[18429.29,2801.25,0.17924512606827214,2,2,2]],[[2024,7,10,16,null,null,null,[-14400]],[18647.45,3019.41,0.1932046501032759,2,2,2]],[[2024,7,11,16,null,null,null,[-14400]],[18283.41,2655.369999999999,0.16991062218934677,2,2,2]],[[2024,7,12,16,null,null,null,[-14400]],[18398.45,2770.41,0.17727175000831835,2,2,2]],[[2024,7,15,16,null,null,null,[-14400]],[18472.57,2844.529999999999,0.1820145072574679,2,2,2]],[[2024,7,16,16,null,null,null,[-14400]],[18509.34,2881.2999999999993,0.18436732949237392,2,2,2]],[[2024,7,17,16,null,null,null,[-14400]],[17996.93,2368.8899999999994,0.15157946869856995,2,2,2]],[[2024,7,18,16,null,null,null,[-14400]],[17871.22,2243.1800000000003,0.14353559371488683,2,2,2]],[[2024,7,19,16,null,null,null,[-14400]],[17726.94,2098.899999999998,0.13430346991689282,2,2,2]],[[2024,7,22,16,null,null,null,[-14400]],[18007.57,2379.529999999999,0.1522602962367641,2,2,2]],[[2024,7,23,16,null,null,null,[-14400]],[17997.35,2369.3099999999977,0.15160634346981436,2,2,2]],[[2024,7,24,16,null,null,null,[-14400]],[17342.41,1714.369999999999,0.1096983370915354,2,2,2]],[[2024,7,25,16,null,null,null,[-14400]],[17181.72,1553.6800000000003,0.09941617758848839,2,2,3]],[[2024,7,26,16,null,null,null,[-14400]],[17357.88,1729.8400000000001,0.11068822449904148,2,2,2]],[[2024,7,26,17,15,59,null,[-14400]],[17357.88,1729.8400000000001,0.11068822449904148,2,2,2]]]]],null,-14400,17181.725,\"ナスダック総合指数\",86400,1],[[\"DAX\",\"INDEXDB\"],\"/m/0877z\",null,[[[1],[[[2024,1,29,20,null,null,null,[7200]],[16941.71,0,0,2,2,2]],[[2024,1,30,20,null,null,null,[7200]],[16972.34,30.63000000000102,0.001807963895026005,2,2,4]],[[2024,1,31,20,null,null,null,[7200]],[16903.76,-37.95000000000073,-0.002240033621163432,2,2,4]],[[2024,2,1,20,null,null,null,[7200]],[16859.04,-82.66999999999825,-0.00487967271308494,2,2,4]],[[2024,2,2,20,null,null,null,[7200]],[16918.21,-23.5,-0.0013871090934740355,2,2,4]],[[2024,2,5,20,null,null,null,[7200]],[16904.06,-37.64999999999782,-0.0022223258455018897,2,2,4]],[[2024,2,6,20,null,null,null,[7200]],[17033.24,91.53000000000247,0.005402642354284336,2,2,4]],[[2024,2,7,20,null,null,null,[7200]],[16921.96,-19.75,-0.001165761897706902,2,2,4]],[[2024,2,8,20,null,null,null,[7200]],[16963.83,22.12000000000262,0.001305653325431885,2,2,4]],[[2024,2,9,20,null,null,null,[7200]],[16926.5,-15.209999999999127,-8.977842260314412E-4,2,2,5]],[[2024,2,12,20,null,null,null,[7200]],[17037.35,95.63999999999942,0.005645238880844934,2,2,4]],[[2024,2,13,20,null,null,null,[7200]],[16880.83,-60.87999999999738,-0.0035934979408806657,2,2,4]],[[2024,2,14,20,null,null,null,[7200]],[16945.48,3.7700000000004366,2.225277141445838E-4,2,2,5]],[[2024,2,15,20,null,null,null,[7200]],[17046.69,104.97999999999956,0.006196540963102282,2,2,4]],[[2024,2,16,20,null,null,null,[7200]],[17117.44,175.72999999999956,0.010372624723242197,2,2,3]],[[2024,2,19,20,null,null,null,[7200]],[17092.26,150.54999999999927,0.00888635208606447,2,2,4]],[[2024,2,20,20,null,null,null,[7200]],[17068.43,126.72000000000116,0.007479764439363038,2,2,4]],[[2024,2,21,20,null,null,null,[7200]],[17118.12,176.40999999999985,0.010412762348074655,2,2,3]],[[2024,2,22,20,null,null,null,[7200]],[17370.45,428.7400000000016,0.025306772456853626,2,2,3]],[[2024,2,23,20,null,null,null,[7200]],[17419.33,477.6200000000026,0.02819195937127968,2,2,3]],[[2024,2,26,20,null,null,null,[7200]],[17423.23,481.52000000000044,0.028422160454877367,2,2,3]],[[2024,2,27,20,null,null,null,[7200]],[17556.49,614.7800000000025,0.03628795440365834,2,2,3]],[[2024,2,28,20,null,null,null,[7200]],[17601.22,659.510000000002,0.038928183754768676,2,2,3]],[[2024,2,29,20,null,null,null,[7200]],[17678.19,736.4799999999996,0.043471408730287535,2,2,3]],[[2024,3,1,20,null,null,null,[7200]],[17735.07,793.3600000000006,0.04682880299568347,2,2,3]],[[2024,3,4,20,null,null,null,[7200]],[17716.17,774.4599999999991,0.045713213129017036,2,2,3]],[[2024,3,5,20,null,null,null,[7200]],[17698.4,756.6900000000023,0.044664322550675366,2,2,3]],[[2024,3,6,20,null,null,null,[7200]],[17716.71,775,0.04574508712520755,2,2,3]],[[2024,3,7,20,null,null,null,[7200]],[17842.85,901.1399999999994,0.05319061653162517,2,2,3]],[[2024,3,8,20,null,null,null,[7200]],[17814.51,872.7999999999993,0.051517821990814346,2,2,3]],[[2024,3,11,20,null,null,null,[7200]],[17746.27,804.5600000000013,0.047489893287041354,2,2,3]],[[2024,3,12,20,null,null,null,[7200]],[17965.11,1023.4000000000015,0.06040712537282255,2,2,3]],[[2024,3,13,20,null,null,null,[7200]],[17961.38,1019.6700000000019,0.06018695869543286,2,2,3]],[[2024,3,14,20,null,null,null,[7200]],[17942.04,1000.3300000000017,0.059045397424463165,2,2,3]],[[2024,3,15,20,null,null,null,[7200]],[17936.65,994.9400000000023,0.058727247721747235,2,2,3]],[[2024,3,18,20,null,null,null,[7200]],[17932.68,990.9700000000012,0.05849291482382836,2,2,3]],[[2024,3,19,20,null,null,null,[7200]],[17987.49,1045.7800000000025,0.06172812543716086,2,2,3]],[[2024,3,20,20,null,null,null,[7200]],[18015.13,1073.420000000002,0.06335960183476178,2,2,3]],[[2024,3,21,20,null,null,null,[7200]],[18179.25,1237.5400000000009,0.0730469356399089,2,2,3]],[[2024,3,22,20,null,null,null,[7200]],[18205.94,1264.2299999999996,0.0746223374145821,2,2,3]],[[2024,3,25,20,null,null,null,[7200]],[18261.31,1319.6000000000022,0.07789060254248256,2,2,3]],[[2024,3,26,20,null,null,null,[7200]],[18384.35,1442.6399999999994,0.08515315160039923,2,2,3]],[[2024,3,27,20,null,null,null,[7200]],[18477.09,1535.380000000001,0.0906272153165177,2,2,3]],[[2024,3,28,20,null,null,null,[7200]],[18492.49,1550.7800000000025,0.09153621446713481,2,2,3]],[[2024,4,2,20,null,null,null,[7200]],[18283.13,1341.420000000002,0.0791785480922529,2,2,3]],[[2024,4,3,20,null,null,null,[7200]],[18367.72,1426.010000000002,0.08417155056957074,2,2,3]],[[2024,4,4,20,null,null,null,[7200]],[18403.13,1461.420000000002,0.08626165835680118,2,2,3]],[[2024,4,5,20,null,null,null,[7200]],[18175.04,1233.3300000000017,0.07279843652146105,2,2,3]],[[2024,4,8,20,null,null,null,[7200]],[18318.97,1377.260000000002,0.08129403702459799,2,2,3]],[[2024,4,9,20,null,null,null,[7200]],[18076.69,1134.9799999999996,0.0669932374004749,2,2,3]],[[2024,4,10,20,null,null,null,[7200]],[18097.3,1155.5900000000001,0.0682097615884111,2,2,3]],[[2024,4,11,20,null,null,null,[7200]],[17954.48,1012.7700000000004,0.05977967985522126,2,2,3]],[[2024,4,12,20,null,null,null,[7200]],[17930.32,988.6100000000006,0.058353613655292215,2,2,3]],[[2024,4,15,20,null,null,null,[7200]],[18026.58,1084.8700000000026,0.06403544860583747,2,2,3]],[[2024,4,16,20,null,null,null,[7200]],[17766.23,824.5200000000004,0.048668050627711165,2,2,3]],[[2024,4,17,20,null,null,null,[7200]],[17770.02,828.3100000000013,0.0488917588602332,2,2,3]],[[2024,4,18,20,null,null,null,[7200]],[17837.4,895.6900000000023,0.05286892527377711,2,2,3]],[[2024,4,19,20,null,null,null,[7200]],[17737.36,795.6500000000015,0.04696397234989865,2,2,3]],[[2024,4,22,20,null,null,null,[7200]],[17860.8,919.0900000000001,0.05425013177536389,2,2,3]],[[2024,4,23,20,null,null,null,[7200]],[18137.65,1195.9400000000023,0.07059145741486558,2,2,3]],[[2024,4,24,20,null,null,null,[7200]],[18088.7,1146.9900000000016,0.06770213868611856,2,2,3]],[[2024,4,25,20,null,null,null,[7200]],[17917.28,975.5699999999997,0.05758391567321125,2,2,3]],[[2024,4,26,20,null,null,null,[7200]],[18161.01,1219.2999999999993,0.07197030287969747,2,2,3]],[[2024,4,29,20,null,null,null,[7200]],[18118.32,1176.6100000000006,0.0694504864030845,2,2,3]],[[2024,4,30,20,null,null,null,[7200]],[17932.17,990.4599999999991,0.05846281160520391,2,2,3]],[[2024,5,2,20,null,null,null,[7200]],[17896.5,954.7900000000009,0.05635735707906704,2,2,3]],[[2024,5,3,20,null,null,null,[7200]],[18001.6,1059.8899999999994,0.06256098115243382,2,2,3]],[[2024,5,6,20,null,null,null,[7200]],[18175.21,1233.5,0.07280847092766905,2,2,3]],[[2024,5,7,20,null,null,null,[7200]],[18430.05,1488.3400000000001,0.08785063609281472,2,2,3]],[[2024,5,8,20,null,null,null,[7200]],[18498.38,1556.670000000002,0.09188387712928636,2,2,3]],[[2024,5,9,20,4,null,null,[7200]],[18686.6,1744.8899999999994,0.10299373557923017,2,2,2]],[[2024,5,10,20,null,null,null,[7200]],[18772.85,1831.1399999999994,0.10808472108187424,2,2,2]],[[2024,5,13,20,null,null,null,[7200]],[18742.22,1800.510000000002,0.10627675718684844,2,2,2]],[[2024,5,14,20,null,null,null,[7200]],[18716.42,1774.7099999999991,0.1047538884799704,2,2,2]],[[2024,5,15,20,null,null,null,[7200]],[18869.36,1927.6500000000015,0.1137813125121373,2,2,2]],[[2024,5,16,20,null,null,null,[7200]],[18738.81,1797.1000000000022,0.10607547880349753,2,2,2]],[[2024,5,17,20,null,null,null,[7200]],[18704.42,1762.7099999999991,0.10404557745351557,2,2,2]],[[2024,5,20,20,4,null,null,[7200]],[18768.96,1827.25,0.10785511025746516,2,2,2]],[[2024,5,21,20,null,null,null,[7200]],[18726.76,1785.0499999999993,0.10536421648109898,2,2,2]],[[2024,5,22,20,null,null,null,[7200]],[18680.2,1738.4900000000016,0.10261596969845439,2,2,2]],[[2024,5,23,20,null,null,null,[7200]],[18691.32,1749.6100000000006,0.10327233791630247,2,2,2]],[[2024,5,24,20,null,null,null,[7200]],[18693.37,1751.6599999999999,0.10339334104998846,2,2,2]],[[2024,5,27,20,null,null,null,[7200]],[18774.71,1833,0.10819450929097477,2,2,2]],[[2024,5,28,20,null,null,null,[7200]],[18677.87,1736.1599999999999,0.1024784393074843,2,2,2]],[[2024,5,29,20,null,null,null,[7200]],[18473.29,1531.5800000000017,0.09040291682480704,2,2,3]],[[2024,5,30,20,4,null,null,[7200]],[18496.79,1555.0800000000017,0.09179002591828109,2,2,3]],[[2024,5,31,20,null,null,null,[7200]],[18497.94,1556.2299999999996,0.09185790572498288,2,2,3]],[[2024,6,3,20,null,null,null,[7200]],[18608.16,1666.4500000000007,0.09836374250297053,2,2,3]],[[2024,6,4,20,null,null,null,[7200]],[18405.64,1463.9300000000003,0.08640981341316788,2,2,3]],[[2024,6,5,20,null,null,null,[7200]],[18575.94,1634.2299999999996,0.09646192739693925,2,2,3]],[[2024,6,6,20,null,null,null,[7200]],[18652.67,1710.9599999999991,0.10099098615192913,2,2,2]],[[2024,6,7,20,null,null,null,[7200]],[18557.27,1615.5600000000013,0.09535991349161338,2,2,3]],[[2024,6,10,20,null,null,null,[7200]],[18494.89,1553.1800000000003,0.09167787667242565,2,2,3]],[[2024,6,11,20,null,null,null,[7200]],[18369.94,1428.2299999999996,0.08430258810946473,2,2,3]],[[2024,6,12,20,null,null,null,[7200]],[18630.86,1689.1500000000015,0.09970363086134762,2,2,3]],[[2024,6,13,20,null,null,null,[7200]],[18265.68,1323.9700000000012,0.0781485458079498,2,2,3]],[[2024,6,14,20,null,null,null,[7200]],[18002.02,1060.3100000000013,0.06258577203835984,2,2,3]],[[2024,6,17,20,null,null,null,[7200]],[18068.21,1126.5,0.06649269760844685,2,2,3]],[[2024,6,18,20,null,null,null,[7200]],[18131.97,1190.260000000002,0.07025619019567694,2,2,3]],[[2024,6,19,20,null,null,null,[7200]],[18067.91,1126.2000000000007,0.06647498983278552,2,2,3]],[[2024,6,20,20,null,null,null,[7200]],[18254.18,1312.4700000000012,0.07746974774093059,2,2,3]],[[2024,6,21,20,null,null,null,[7200]],[18163.52,1221.8100000000013,0.07211845793606439,2,2,3]],[[2024,6,24,20,null,null,null,[7200]],[18325.58,1383.8700000000026,0.0816841983483369,2,2,3]],[[2024,6,25,20,null,null,null,[7200]],[18177.62,1235.9099999999999,0.07295072339214873,2,2,3]],[[2024,6,26,20,null,null,null,[7200]],[18155.24,1213.5300000000025,0.07162972332781062,2,2,3]],[[2024,6,27,20,null,null,null,[7200]],[18210.55,1268.8400000000001,0.07489444690057853,2,2,3]],[[2024,6,28,20,null,null,null,[7200]],[18235.45,1293.7400000000016,0.07636419228047238,2,2,3]],[[2024,7,1,20,null,null,null,[7200]],[18290.66,1348.9500000000007,0.07962301326135324,2,2,3]],[[2024,7,2,20,null,null,null,[7200]],[18164.06,1222.3500000000022,0.07215033193225491,2,2,3]],[[2024,7,3,20,null,null,null,[7200]],[18374.53,1432.8199999999997,0.0845735170770837,2,2,3]],[[2024,7,4,20,null,null,null,[7200]],[18450.48,1508.7700000000004,0.0890565356153541,2,2,3]],[[2024,7,5,20,null,null,null,[7200]],[18475.45,1533.7400000000016,0.0905304128095689,2,2,3]],[[2024,7,8,20,null,null,null,[7200]],[18472.05,1530.3400000000001,0.09032972468540662,2,2,3]],[[2024,7,9,20,null,null,null,[7200]],[18236.19,1294.4799999999996,0.07640787146043697,2,2,3]],[[2024,7,10,20,null,null,null,[7200]],[18407.22,1465.510000000002,0.08650307436498453,2,2,3]],[[2024,7,11,20,null,null,null,[7200]],[18534.56,1592.8500000000022,0.09401943487404768,2,2,3]],[[2024,7,12,20,null,null,null,[7200]],[18748.18,1806.4700000000012,0.10662855166332096,2,2,2]],[[2024,7,15,20,null,null,null,[7200]],[18590.89,1649.1800000000003,0.09734436488406427,2,2,3]],[[2024,7,16,20,null,null,null,[7200]],[18518.03,1576.3199999999997,0.09304373643510601,2,2,3]],[[2024,7,17,20,null,null,null,[7200]],[18437.3,1495.5900000000001,0.08827857400463118,2,2,3]],[[2024,7,18,20,null,null,null,[7200]],[18354.76,1413.0499999999993,0.08340657466099935,2,2,3]],[[2024,7,19,20,null,null,null,[7200]],[18171.93,1230.2200000000012,0.07261486591377146,2,2,3]],[[2024,7,22,20,null,null,null,[7200]],[18407.07,1465.3600000000006,0.08649422047715376,2,2,3]],[[2024,7,23,20,null,null,null,[7200]],[18557.7,1615.9900000000016,0.09538529463672803,2,2,3]],[[2024,7,24,20,null,null,null,[7200]],[18387.46,1445.75,0.0853367222080888,2,2,3]],[[2024,7,25,20,null,null,null,[7200]],[18298.72,1357.010000000002,0.08009876216745547,2,2,3]],[[2024,7,26,20,null,null,null,[7200]],[18417.55,1475.8400000000001,0.08711281210692429,2,2,3]]]]],null,7200,18298.72,\"ドイツ株価指数\",86400,1],[[\"TOPIX\",\"INDEXTOPIX\"],\"/m/02jyhg\",null,[[[1],[[[2024,1,29,15,null,null,null,[32400]],[2529.48,0,0,2,2,2]],[[2024,1,30,15,null,null,null,[32400]],[2526.93,-2.550000000000182,-0.0010081123392951048,2,2,4]],[[2024,1,31,15,null,null,null,[32400]],[2551.1,21.61999999999989,0.008547211284532745,2,2,4]],[[2024,2,1,15,null,null,null,[32400]],[2534.04,4.559999999999945,0.0018027420655628609,2,2,4]],[[2024,2,2,15,null,null,null,[32400]],[2539.68,10.199999999999818,0.00403244935718006,2,2,4]],[[2024,2,5,15,null,null,null,[32400]],[2556.71,27.230000000000018,0.01076505843098187,2,2,3]],[[2024,2,6,15,null,null,null,[32400]],[2539.25,9.769999999999982,0.003862453943102923,2,2,4]],[[2024,2,7,15,null,null,null,[32400]],[2549.95,20.4699999999998,0.008092572386419264,2,2,4]],[[2024,2,8,15,null,null,null,[32400]],[2562.63,33.15000000000009,0.013105460410835464,2,2,3]],[[2024,2,9,15,null,null,null,[32400]],[2557.88,28.40000000000009,0.011227604092540795,2,2,3]],[[2024,2,13,15,null,null,null,[32400]],[2612.03,82.55000000000018,0.03263516612110006,2,2,3]],[[2024,2,14,15,null,null,null,[32400]],[2584.59,55.11000000000013,0.02178708667394094,2,2,3]],[[2024,2,15,15,null,null,null,[32400]],[2591.85,62.36999999999989,0.024657241804639646,2,2,3]],[[2024,2,16,15,null,null,null,[32400]],[2624.73,95.25,0.037655960908961526,2,2,3]],[[2024,2,19,15,null,null,null,[32400]],[2639.69,110.21000000000004,0.04357021996615907,2,2,3]],[[2024,2,20,15,null,null,null,[32400]],[2632.3,102.82000000000016,0.04064867087306488,2,2,3]],[[2024,2,21,15,null,null,null,[32400]],[2627.3,97.82000000000016,0.03867198001170207,2,2,3]],[[2024,2,22,15,null,null,null,[32400]],[2660.71,131.23000000000002,0.05188022834732831,2,2,3]],[[2024,2,26,15,null,null,null,[32400]],[2673.62,144.13999999999987,0.05698404415136703,2,2,3]],[[2024,2,27,15,null,null,null,[32400]],[2678.46,148.98000000000002,0.058897480905166284,2,2,3]],[[2024,2,28,15,null,null,null,[32400]],[2674.95,145.4699999999998,0.05750984392048951,2,2,3]],[[2024,2,29,15,null,null,null,[32400]],[2675.73,146.25,0.057818207694862185,2,2,3]],[[2024,3,1,15,null,null,null,[32400]],[2709.42,179.94000000000005,0.07113715071872481,2,2,3]],[[2024,3,4,15,null,null,null,[32400]],[2706.28,176.80000000000018,0.06989578885778902,2,2,3]],[[2024,3,5,15,null,null,null,[32400]],[2719.93,190.44999999999982,0.07529215490930935,2,2,3]],[[2024,3,6,15,null,null,null,[32400]],[2730.67,201.19000000000005,0.07953808687951676,2,2,3]],[[2024,3,7,15,null,null,null,[32400]],[2718.54,189.05999999999995,0.07474263484985054,2,2,3]],[[2024,3,8,15,null,null,null,[32400]],[2726.8,197.32000000000016,0.07800812815282199,2,2,3]],[[2024,3,11,15,null,null,null,[32400]],[2666.83,137.3499999999999,0.05429969796163635,2,2,3]],[[2024,3,12,15,null,null,null,[32400]],[2657.24,127.75999999999976,0.05050840488954242,2,2,3]],[[2024,3,13,15,null,null,null,[32400]],[2648.51,119.0300000000002,0.04705710264560313,2,2,3]],[[2024,3,14,15,null,null,null,[32400]],[2661.59,132.11000000000013,0.052228125938928206,2,2,3]],[[2024,3,15,15,null,null,null,[32400]],[2670.8,141.32000000000016,0.05586919050555852,2,2,3]],[[2024,3,18,15,null,null,null,[32400]],[2721.99,192.50999999999976,0.07610655154419081,2,2,3]],[[2024,3,19,15,null,null,null,[32400]],[2750.97,221.48999999999978,0.08756345177664966,2,2,3]],[[2024,3,21,15,null,null,null,[32400]],[2796.21,266.73,0.10544855069026046,2,2,2]],[[2024,3,22,15,null,null,null,[32400]],[2813.22,283.7399999999998,0.11217325300061665,2,2,2]],[[2024,3,25,15,null,null,null,[32400]],[2777.64,248.15999999999985,0.09810712083115891,2,2,3]],[[2024,3,26,15,null,null,null,[32400]],[2780.8,251.32000000000016,0.09935638945554033,2,2,3]],[[2024,3,27,15,null,null,null,[32400]],[2799.28,269.8000000000002,0.10666223887913728,2,2,2]],[[2024,3,28,15,null,null,null,[32400]],[2750.81,221.32999999999993,0.08750019766908611,2,2,3]],[[2024,3,29,15,null,null,null,[32400]],[2768.62,239.13999999999987,0.09454117051726041,2,2,3]],[[2024,4,1,15,null,null,null,[32400]],[2721.22,191.73999999999978,0.07580214115154094,2,2,3]],[[2024,4,2,15,null,null,null,[32400]],[2714.45,184.9699999999998,0.0731257017252557,2,2,3]],[[2024,4,3,15,null,null,null,[32400]],[2706.51,177.0300000000002,0.06998671663741172,2,2,3]],[[2024,4,4,15,null,null,null,[32400]],[2732,202.51999999999998,0.08006388664863924,2,2,3]],[[2024,4,5,15,null,null,null,[32400]],[2702.62,173.13999999999987,0.06844885114727133,2,2,3]],[[2024,4,8,15,null,null,null,[32400]],[2728.32,198.84000000000015,0.07860904217467628,2,2,3]],[[2024,4,9,15,null,null,null,[32400]],[2754.69,225.21000000000004,0.08903410977750369,2,2,3]],[[2024,4,10,15,null,null,null,[32400]],[2742.79,213.30999999999995,0.08432958552746017,2,2,3]],[[2024,4,11,15,null,null,null,[32400]],[2746.96,217.48000000000002,0.08597814570583678,2,2,3]],[[2024,4,12,15,null,null,null,[32400]],[2759.64,230.15999999999985,0.0909910337302528,2,2,3]],[[2024,4,15,15,null,null,null,[32400]],[2753.2,223.7199999999998,0.08844505590081748,2,2,3]],[[2024,4,16,15,null,null,null,[32400]],[2697.11,167.6300000000001,0.0662705378180496,2,2,3]],[[2024,4,17,15,null,null,null,[32400]],[2663.15,133.67000000000007,0.052844853487673385,2,2,3]],[[2024,4,18,15,null,null,null,[32400]],[2677.45,147.9699999999998,0.05849818935117091,2,2,3]],[[2024,4,19,15,null,null,null,[32400]],[2626.32,96.84000000000015,0.03828454860287496,2,2,3]],[[2024,4,22,15,null,null,null,[32400]],[2662.46,132.98000000000002,0.05257207014880529,2,2,3]],[[2024,4,23,15,null,null,null,[32400]],[2666.23,136.75,0.05406249505827285,2,2,3]],[[2024,4,24,15,null,null,null,[32400]],[2710.73,181.25,0.07165504372440186,2,2,3]],[[2024,4,25,15,null,null,null,[32400]],[2663.53,134.05000000000018,0.052995081993137,2,2,3]],[[2024,4,26,15,null,null,null,[32400]],[2686.48,157,0.06206809304679223,2,2,3]],[[2024,4,30,15,null,null,null,[32400]],[2743.17,213.69000000000005,0.08447981403292379,2,2,3]],[[2024,5,1,15,null,null,null,[32400]],[2729.4,199.92000000000007,0.0790360074007306,2,2,3]],[[2024,5,2,15,null,null,null,[32400]],[2728.53,199.05000000000018,0.07869206319085353,2,2,3]],[[2024,5,7,15,null,null,null,[32400]],[2746.22,216.73999999999978,0.085685595458355,2,2,3]],[[2024,5,8,15,null,null,null,[32400]],[2706.43,176.94999999999982,0.06995508958362977,2,2,3]],[[2024,5,9,15,null,null,null,[32400]],[2713.46,183.98000000000002,0.07273431693470596,2,2,3]],[[2024,5,10,15,null,null,null,[32400]],[2728.21,198.73000000000002,0.07856555497572625,2,2,3]],[[2024,5,13,15,null,null,null,[32400]],[2724.08,194.5999999999999,0.07693280832424051,2,2,3]],[[2024,5,14,15,null,null,null,[32400]],[2730.95,201.4699999999998,0.07964878156775297,2,2,3]],[[2024,5,15,15,null,null,null,[32400]],[2730.88,201.4000000000001,0.07962110789569402,2,2,3]],[[2024,5,16,15,null,null,null,[32400]],[2737.54,208.05999999999995,0.08225406012302922,2,2,3]],[[2024,5,17,15,null,null,null,[32400]],[2745.62,216.13999999999987,0.0854483925549915,2,2,3]],[[2024,5,20,15,null,null,null,[32400]],[2768.04,238.55999999999995,0.09431187437734236,2,2,3]],[[2024,5,21,15,null,null,null,[32400]],[2759.72,230.23999999999978,0.09102266078403458,2,2,3]],[[2024,5,22,15,null,null,null,[32400]],[2737.36,207.8800000000001,0.08218289925202021,2,2,3]],[[2024,5,23,15,null,null,null,[32400]],[2754.75,225.26999999999998,0.08905783006784002,2,2,3]],[[2024,5,24,15,null,null,null,[32400]],[2742.54,213.05999999999995,0.08423075098439203,2,2,3]],[[2024,5,27,15,null,null,null,[32400]],[2766.36,236.8800000000001,0.09364770624792451,2,2,3]],[[2024,5,28,15,null,null,null,[32400]],[2768.5,239.01999999999998,0.09449372993658775,2,2,3]],[[2024,5,29,15,null,null,null,[32400]],[2741.62,212.13999999999987,0.08386703986590124,2,2,3]],[[2024,5,30,15,null,null,null,[32400]],[2726.2,196.7199999999998,0.0777709252494583,2,2,3]],[[2024,5,31,15,null,null,null,[32400]],[2772.49,243.00999999999976,0.09607112924395518,2,2,3]],[[2024,6,3,15,null,null,null,[32400]],[2798.07,268.59000000000015,0.10618387969068747,2,2,2]],[[2024,6,4,15,null,null,null,[32400]],[2787.48,258,0.10199724844632098,2,2,2]],[[2024,6,5,15,null,null,null,[32400]],[2748.22,218.73999999999978,0.08647627180290011,2,2,3]],[[2024,6,6,15,null,null,null,[32400]],[2757.23,227.75,0.09003826873507598,2,2,3]],[[2024,6,7,15,null,null,null,[32400]],[2755.03,225.55000000000018,0.08916852475607642,2,2,3]],[[2024,6,10,15,null,null,null,[32400]],[2782.49,253.00999999999976,0.1000245109666808,2,2,2]],[[2024,6,11,15,null,null,null,[32400]],[2776.8,247.32000000000016,0.09777503676645008,2,2,3]],[[2024,6,12,15,null,null,null,[32400]],[2756.44,226.96000000000004,0.08972595157898067,2,2,3]],[[2024,6,13,15,null,null,null,[32400]],[2731.78,202.30000000000018,0.07997691225073936,2,2,3]],[[2024,6,14,15,null,null,null,[32400]],[2746.61,217.1300000000001,0.08583977734554142,2,2,3]],[[2024,6,17,15,null,null,null,[32400]],[2700.01,170.5300000000002,0.06741701851764006,2,2,3]],[[2024,6,18,15,null,null,null,[32400]],[2715.76,186.2800000000002,0.07364359473093292,2,2,3]],[[2024,6,19,15,null,null,null,[32400]],[2728.64,199.15999999999985,0.07873555038980339,2,2,3]],[[2024,6,20,15,null,null,null,[32400]],[2725.54,196.05999999999995,0.07751000205575848,2,2,3]],[[2024,6,21,15,null,null,null,[32400]],[2724.69,195.21000000000004,0.07717396460932684,2,2,3]],[[2024,6,24,15,null,null,null,[32400]],[2740.19,210.71000000000004,0.08330170627955154,2,2,3]],[[2024,6,25,15,null,null,null,[32400]],[2787.37,257.8899999999999,0.10195376124737095,2,2,2]],[[2024,6,26,15,null,null,null,[32400]],[2802.95,273.4699999999998,0.10811312997137744,2,2,2]],[[2024,6,27,15,null,null,null,[32400]],[2793.7,264.2199999999998,0.10445625187785623,2,2,2]],[[2024,6,28,15,null,null,null,[32400]],[2809.63,280.1500000000001,0.11075398896215827,2,2,2]],[[2024,7,1,15,null,null,null,[32400]],[2824.28,294.8000000000002,0.11654569318595133,2,2,2]],[[2024,7,2,15,null,null,null,[32400]],[2856.62,327.1399999999999,0.12933092967724585,2,2,2]],[[2024,7,3,15,null,null,null,[32400]],[2872.18,342.6999999999998,0.1354823916378069,2,2,2]],[[2024,7,4,15,null,null,null,[32400]],[2898.47,368.9899999999998,0.14587583218685254,2,2,2]],[[2024,7,5,15,null,null,null,[32400]],[2884.18,354.6999999999998,0.14022644970507764,2,2,2]],[[2024,7,8,15,null,null,null,[32400]],[2867.61,338.1300000000001,0.13367569619052141,2,2,2]],[[2024,7,9,15,null,null,null,[32400]],[2895.55,366.07000000000016,0.14472144472381682,2,2,2]],[[2024,7,10,15,null,null,null,[32400]],[2909.2,379.7199999999998,0.15011781077533715,2,2,2]],[[2024,7,11,15,null,null,null,[32400]],[2929.17,399.69000000000005,0.15801271407562031,2,2,2]],[[2024,7,12,15,null,null,null,[32400]],[2894.56,365.0799999999999,0.1443300599332669,2,2,2]],[[2024,7,16,15,null,null,null,[32400]],[2904.5,375.02,0.14825972136565618,2,2,2]],[[2024,7,17,15,null,null,null,[32400]],[2915.21,385.73,0.15249379319069534,2,2,2]],[[2024,7,18,15,null,null,null,[32400]],[2868.63,339.1500000000001,0.13407894112623941,2,2,2]],[[2024,7,19,15,null,null,null,[32400]],[2860.83,331.3499999999999,0.13099530338251336,2,2,2]],[[2024,7,22,15,null,null,null,[32400]],[2827.53,298.0500000000002,0.11783054224583717,2,2,2]],[[2024,7,23,15,null,null,null,[32400]],[2833.39,303.90999999999985,0.12014722393535425,2,2,2]],[[2024,7,24,15,null,null,null,[32400]],[2793.12,263.6399999999999,0.10422695573793818,2,2,2]],[[2024,7,25,15,null,null,null,[32400]],[2709.86,180.3800000000001,0.07131109951452477,2,2,3]],[[2024,7,26,15,null,null,null,[32400]],[2699.54,170.05999999999995,0.06723120957667186,2,2,3]],[[2024,7,26,15,null,1,null,[32400]],[2699.54,170.05999999999995,0.06723120957667186,2,2,3]]]]],null,32400,2709.86,\"東証株価指数\",86400,1],[[\"NIFTY_50\",\"INDEXNSE\"],\"/m/04t5sp\",null,[[[1],[[[2024,1,29,15,30,null,null,[19800]],[21737.6,0,0,2,2,2]],[[2024,1,30,15,30,null,null,[19800]],[21522.1,-215.5,-0.009913697924333874,2,2,4]],[[2024,1,31,15,30,null,null,[19800]],[21725.7,-11.899999999997817,-5.474385396730926E-4,2,2,5]],[[2024,2,1,15,30,null,null,[19800]],[21697.45,-40.14999999999782,-0.0018470300309140761,2,2,4]],[[2024,2,2,15,30,null,null,[19800]],[21853.8,116.20000000000073,0.005345576328573565,2,2,4]],[[2024,2,5,15,30,null,null,[19800]],[21771.7,34.10000000000218,0.0015687104372148805,2,2,4]],[[2024,2,6,15,30,null,null,[19800]],[21929.4,191.8000000000029,0.008823421168850421,2,2,4]],[[2024,2,7,15,30,null,null,[19800]],[21930.5,192.90000000000146,0.008874024731341154,2,2,4]],[[2024,2,8,15,30,null,null,[19800]],[21717.95,-19.649999999997817,-9.03963639040088E-4,2,2,5]],[[2024,2,9,15,30,null,null,[19800]],[21782.5,44.900000000001455,0.0020655454143972407,2,2,4]],[[2024,2,12,15,30,null,null,[19800]],[21616.05,-121.54999999999927,-0.005591693655233296,2,2,4]],[[2024,2,13,15,30,null,null,[19800]],[21743.25,5.650000000001455,2.5991829824826364E-4,2,2,5]],[[2024,2,14,15,30,null,null,[19800]],[21840.05,102.45000000000073,0.004713031797438574,2,2,4]],[[2024,2,15,15,30,null,null,[19800]],[21910.75,173.15000000000146,0.007965460768438166,2,2,4]],[[2024,2,16,15,30,null,null,[19800]],[22040.7,303.1000000000022,0.013943581628146722,2,2,3]],[[2024,2,19,15,30,null,null,[19800]],[22122.25,384.65000000000146,0.017695145738260042,2,2,3]],[[2024,2,20,15,30,null,null,[19800]],[22196.95,459.3500000000022,0.02113158766377163,2,2,3]],[[2024,2,21,15,30,null,null,[19800]],[22055.05,317.4500000000007,0.014603728102458448,2,2,3]],[[2024,2,22,15,30,null,null,[19800]],[22217.45,479.8500000000022,0.02207465405564562,2,2,3]],[[2024,2,23,15,30,null,null,[19800]],[22212.7,475.1000000000022,0.02185613867216262,2,2,3]],[[2024,2,26,15,30,null,null,[19800]],[22122.05,384.4500000000007,0.017685945090534408,2,2,3]],[[2024,2,27,15,30,null,null,[19800]],[22198.35,460.75,0.02119599219785073,2,2,3]],[[2024,2,28,15,30,null,null,[19800]],[21951.15,213.5500000000029,0.00982399160900941,2,2,4]],[[2024,2,29,15,30,null,null,[19800]],[21982.8,245.20000000000073,0.01127999411158549,2,2,3]],[[2024,3,1,15,30,null,null,[19800]],[22338.75,601.1500000000015,0.027654846901221913,2,2,3]],[[2024,3,4,15,30,null,null,[19800]],[22405.6,668,0.03073016340350361,2,2,3]],[[2024,3,5,15,30,null,null,[19800]],[22356.3,618.7000000000007,0.028462203739143273,2,2,3]],[[2024,3,6,15,30,null,null,[19800]],[22474.05,736.4500000000007,0.0338790850875902,2,2,3]],[[2024,3,7,15,30,null,null,[19800]],[22493.55,755.9500000000007,0.03477614824083619,2,2,3]],[[2024,3,11,15,30,null,null,[19800]],[22332.65,595.0500000000029,0.027374227145591186,2,2,3]],[[2024,3,12,15,30,null,null,[19800]],[22335.7,598.1000000000022,0.02751453702340655,2,2,3]],[[2024,3,13,15,30,null,null,[19800]],[21997.7,260.1000000000022,0.011965442367142748,2,2,3]],[[2024,3,14,15,30,null,null,[19800]],[22146.65,409.0500000000029,0.018817624760783296,2,2,3]],[[2024,3,15,15,30,null,null,[19800]],[22023.35,285.75,0.013145425437950832,2,2,3]],[[2024,3,18,15,30,null,null,[19800]],[22055.7,318.1000000000022,0.014633630207566714,2,2,3]],[[2024,3,19,15,30,null,null,[19800]],[21817.45,79.85000000000218,0.0036733586044458534,2,2,4]],[[2024,3,20,15,30,null,null,[19800]],[21839.1,101.5,0.0046693287207419405,2,2,4]],[[2024,3,21,15,30,null,null,[19800]],[22011.95,274.3500000000022,0.01262098851759174,2,2,3]],[[2024,3,22,15,30,null,null,[19800]],[22096.75,359.15000000000146,0.016522063153246058,2,2,3]],[[2024,3,26,15,30,null,null,[19800]],[22004.7,267.1000000000022,0.012287465037538744,2,2,3]],[[2024,3,27,15,30,null,null,[19800]],[22123.65,386.0500000000029,0.017759550272339306,2,2,3]],[[2024,3,28,15,30,null,null,[19800]],[22326.9,589.3000000000029,0.02710970852348019,2,2,3]],[[2024,4,1,15,30,null,null,[19800]],[22462,724.4000000000015,0.033324746062122844,2,2,3]],[[2024,4,2,15,30,null,null,[19800]],[22453.3,715.7000000000007,0.03292451788605921,2,2,3]],[[2024,4,3,15,30,null,null,[19800]],[22434.65,697.0500000000029,0.03206655748564712,2,2,3]],[[2024,4,4,15,30,null,null,[19800]],[22514.65,777.0500000000029,0.03574681657588708,2,2,3]],[[2024,4,5,15,30,null,null,[19800]],[22513.7,776.1000000000022,0.03570311349919045,2,2,3]],[[2024,4,8,15,30,null,null,[19800]],[22666.3,928.7000000000007,0.04272320771382309,2,2,3]],[[2024,4,9,15,30,null,null,[19800]],[22642.75,905.1500000000015,0.04163983144413374,2,2,3]],[[2024,4,12,15,30,null,null,[19800]],[22519.4,781.8000000000029,0.03596533195937007,2,2,3]],[[2024,4,15,15,30,null,null,[19800]],[22272.5,534.9000000000015,0.024607132342116953,2,2,3]],[[2024,4,16,15,30,null,null,[19800]],[22147.9,410.3000000000029,0.018875128809068293,2,2,3]],[[2024,4,18,15,30,null,null,[19800]],[21995.85,258.25,0.01188033637568085,2,2,3]],[[2024,4,19,15,30,null,null,[19800]],[22147,409.40000000000146,0.018833725894303027,2,2,3]],[[2024,4,22,15,30,null,null,[19800]],[22336.4,598.8000000000029,0.02754673929044618,2,2,3]],[[2024,4,23,15,30,null,null,[19800]],[22368,630.4000000000015,0.029000441631090897,2,2,3]],[[2024,4,24,15,30,null,null,[19800]],[22402.4,664.8000000000029,0.030582953039894143,2,2,3]],[[2024,4,25,15,30,null,null,[19800]],[22570.35,832.75,0.03830919696746651,2,2,3]],[[2024,4,26,15,30,null,null,[19800]],[22419.95,682.3500000000022,0.0313903098778155,2,2,3]],[[2024,4,29,15,30,null,null,[19800]],[22643.4,905.8000000000029,0.041669733549242007,2,2,3]],[[2024,4,30,15,30,null,null,[19800]],[22604.85,867.25,0.03989630870013249,2,2,3]],[[2024,5,2,15,30,null,null,[19800]],[22648.2,910.6000000000022,0.041890549094656365,2,2,3]],[[2024,5,3,15,30,null,null,[19800]],[22475.85,738.25,0.03396189091712057,2,2,3]],[[2024,5,6,15,30,null,null,[19800]],[22442.7,705.1000000000022,0.032436883556602486,2,2,3]],[[2024,5,7,15,30,null,null,[19800]],[22302.5,564.9000000000015,0.025987229500956936,2,2,3]],[[2024,5,8,15,30,null,null,[19800]],[22302.5,564.9000000000015,0.025987229500956936,2,2,3]],[[2024,5,9,15,30,null,null,[19800]],[21957.5,219.90000000000146,0.010116112174297138,2,2,3]],[[2024,5,10,15,30,null,null,[19800]],[22055.2,317.6000000000022,0.014610628588252714,2,2,3]],[[2024,5,13,15,30,null,null,[19800]],[22104.05,366.4500000000007,0.01685788679523042,2,2,3]],[[2024,5,14,15,30,null,null,[19800]],[22217.85,480.25,0.022093055351096718,2,2,3]],[[2024,5,15,15,30,null,null,[19800]],[22200.55,462.9500000000007,0.02129719932283236,2,2,3]],[[2024,5,16,15,30,null,null,[19800]],[22403.85,666.25,0.03064965773590461,2,2,3]],[[2024,5,17,15,30,null,null,[19800]],[22466.1,728.5,0.033513359340497574,2,2,3]],[[2024,5,21,15,30,null,null,[19800]],[22529.05,791.4500000000007,0.03640926321213017,2,2,3]],[[2024,5,22,15,30,null,null,[19800]],[22597.8,860.2000000000007,0.03957198586780513,2,2,3]],[[2024,5,23,15,30,null,null,[19800]],[22967.65,1230.050000000003,0.05658628367437081,2,2,3]],[[2024,5,24,15,30,null,null,[19800]],[22957.1,1219.5,0.056100949506845285,2,2,3]],[[2024,5,27,15,30,null,null,[19800]],[22932.45,1194.8500000000022,0.0549669696746652,2,2,3]],[[2024,5,28,15,30,null,null,[19800]],[22888.15,1150.550000000003,0.05292902620344486,2,2,3]],[[2024,5,29,15,30,null,null,[19800]],[22704.7,967.1000000000022,0.044489732077138334,2,2,3]],[[2024,5,30,15,30,null,null,[19800]],[22488.65,751.0500000000029,0.0345507323715591,2,2,3]],[[2024,5,31,15,30,null,null,[19800]],[22530.7,793.1000000000022,0.036485168555866435,2,2,3]],[[2024,6,3,15,30,null,null,[19800]],[23263.9,1526.300000000003,0.07021474311791565,2,2,3]],[[2024,6,4,15,30,null,null,[19800]],[21884.5,146.90000000000146,0.0067578757544531805,2,2,4]],[[2024,6,5,15,30,null,null,[19800]],[22620.35,882.75,0.040609358898866486,2,2,3]],[[2024,6,6,15,30,null,null,[19800]],[22821.4,1083.800000000003,0.0498583100250259,2,2,3]],[[2024,6,7,15,30,null,null,[19800]],[23290.15,1552.550000000003,0.07142232813190062,2,2,3]],[[2024,6,10,15,30,null,null,[19800]],[23259.2,1521.6000000000022,0.06999852789636402,2,2,3]],[[2024,6,11,15,30,null,null,[19800]],[23264.85,1527.25,0.0702584461946121,2,2,3]],[[2024,6,12,15,30,null,null,[19800]],[23322.95,1585.3500000000022,0.07293123435889898,2,2,3]],[[2024,6,13,15,30,null,null,[19800]],[23398.9,1661.300000000003,0.07642518033269556,2,2,3]],[[2024,6,14,15,30,null,null,[19800]],[23465.6,1728,0.07949359634918299,2,2,3]],[[2024,6,18,15,30,null,null,[19800]],[23557.9,1820.300000000003,0.08373969527454747,2,2,3]],[[2024,6,19,15,30,null,null,[19800]],[23516,1778.4000000000015,0.08181215957603423,2,2,3]],[[2024,6,20,15,30,null,null,[19800]],[23567,1829.4000000000015,0.08415832474606219,2,2,3]],[[2024,6,21,15,30,null,null,[19800]],[23501.1,1763.5,0.08112671132047697,2,2,3]],[[2024,6,24,15,30,null,null,[19800]],[23537.85,1800.25,0.08281733034005595,2,2,3]],[[2024,6,25,15,30,null,null,[19800]],[23721.3,1983.7000000000007,0.09125662446636247,2,2,3]],[[2024,6,26,15,30,null,null,[19800]],[23868.8,2131.2000000000007,0.09804210216399238,2,2,3]],[[2024,6,27,15,30,null,null,[19800]],[24044.5,2306.9000000000015,0.10612487119093192,2,2,2]],[[2024,6,28,15,30,null,null,[19800]],[24010.6,2273,0.10456536140144267,2,2,2]],[[2024,7,1,15,30,null,null,[19800]],[24141.95,2404.350000000002,0.1106078867952305,2,2,2]],[[2024,7,2,15,30,null,null,[19800]],[24123.85,2386.25,0.1097752281760636,2,2,2]],[[2024,7,3,15,30,null,null,[19800]],[24286.5,2548.9000000000015,0.11725765493890777,2,2,2]],[[2024,7,4,15,30,null,null,[19800]],[24302.15,2564.550000000003,0.11797760562343604,2,2,2]],[[2024,7,5,15,30,null,null,[19800]],[24323.85,2586.25,0.11897587590166349,2,2,2]],[[2024,7,8,15,30,null,null,[19800]],[24320.55,2582.9500000000007,0.11882406521419112,2,2,2]],[[2024,7,9,15,30,null,null,[19800]],[24433.2,2695.600000000002,0.12400633004563533,2,2,2]],[[2024,7,10,15,30,null,null,[19800]],[24324.45,2586.850000000002,0.11900347784484039,2,2,2]],[[2024,7,11,15,30,null,null,[19800]],[24315.95,2578.350000000002,0.11861245031650239,2,2,2]],[[2024,7,12,15,30,null,null,[19800]],[24502.15,2764.550000000003,0.1271782533490359,2,2,2]],[[2024,7,15,15,30,null,null,[19800]],[24586.7,2849.100000000002,0.13106782717503324,2,2,2]],[[2024,7,16,15,30,null,null,[19800]],[24613,2875.4000000000015,0.1322777123509496,2,2,2]],[[2024,7,18,15,30,null,null,[19800]],[24800.85,3063.25,0.14091942072721922,2,2,2]],[[2024,7,19,15,30,null,null,[19800]],[24530.9,2793.300000000003,0.1285008464595909,2,2,2]],[[2024,7,22,15,30,null,null,[19800]],[24509.25,2771.6500000000015,0.12750487634329463,2,2,2]],[[2024,7,23,15,30,null,null,[19800]],[24479.05,2741.4500000000007,0.12611557853672903,2,2,2]],[[2024,7,24,15,30,null,null,[19800]],[24413.5,2675.9000000000015,0.1231000662446637,2,2,2]],[[2024,7,25,15,30,null,null,[19800]],[24406.1,2668.5,0.12275964227881643,2,2,2]],[[2024,7,26,15,30,null,null,[19800]],[24834.85,3097.25,0.1424835308405712,2,2,2]],[[2024,7,26,15,32,10,null,[19800]],[24834.85,3097.25,0.1424835308405712,2,2,2]]]]],null,19800,24406.1,\"NIFTY 50\",86400,1],[[\"SOX\",\"INDEXNASDAQ\"],\"/m/0637tx\",null,null,null,-14400,5005.358,\"フィラデルフィア半導体株指数\",86400,1],[[\"VIX\",\"INDEXCBOE\"],\"/m/09fld6\",null,[[[1],[[[2024,1,29,15,16,null,null,[-18000]],[13.6,0,0,2,2,2]],[[2024,1,30,15,16,null,null,[-18000]],[13.31,-0.28999999999999915,-0.021323529411764644,2,2,3]],[[2024,1,31,15,16,null,null,[-18000]],[14.35,0.75,0.05514705882352941,2,2,3]],[[2024,2,1,15,16,null,null,[-18000]],[13.88,0.28000000000000114,0.02058823529411773,2,2,3]],[[2024,2,2,15,16,null,null,[-18000]],[13.85,0.25,0.01838235294117647,2,2,3]],[[2024,2,5,15,16,null,null,[-18000]],[13.67,0.07000000000000028,0.0051470588235294325,2,3,4]],[[2024,2,6,15,16,null,null,[-18000]],[13.06,-0.5399999999999991,-0.03970588235294111,2,2,3]],[[2024,2,7,15,16,null,null,[-18000]],[12.83,-0.7699999999999996,-0.0566176470588235,2,2,3]],[[2024,2,8,15,16,null,null,[-18000]],[12.79,-0.8100000000000005,-0.0595588235294118,2,2,3]],[[2024,2,9,15,16,null,null,[-18000]],[12.93,-0.6699999999999999,-0.04926470588235294,2,2,3]],[[2024,2,12,15,16,null,null,[-18000]],[13.93,0.33000000000000007,0.02426470588235295,2,2,3]],[[2024,2,13,15,16,null,null,[-18000]],[15.85,2.25,0.16544117647058823,2,2,2]],[[2024,2,14,15,16,null,null,[-18000]],[14.38,0.7800000000000011,0.057352941176470676,2,2,3]],[[2024,2,15,15,16,null,null,[-18000]],[14.01,0.41000000000000014,0.030147058823529423,2,2,3]],[[2024,2,16,15,16,null,null,[-18000]],[14.24,0.6400000000000006,0.047058823529411806,2,2,3]],[[2024,2,20,15,16,null,null,[-18000]],[15.42,1.8200000000000003,0.13382352941176473,2,2,2]],[[2024,2,21,15,16,null,null,[-18000]],[15.34,1.7400000000000002,0.12794117647058825,2,2,2]],[[2024,2,22,15,16,null,null,[-18000]],[14.54,0.9399999999999995,0.06911764705882349,2,2,3]],[[2024,2,23,15,16,null,null,[-18000]],[13.75,0.15000000000000036,0.011029411764705909,2,2,3]],[[2024,2,26,15,16,null,null,[-18000]],[13.74,0.14000000000000057,0.010294117647058865,2,2,3]],[[2024,2,27,15,16,null,null,[-18000]],[13.43,-0.16999999999999993,-0.012499999999999995,2,2,3]],[[2024,2,28,15,16,null,null,[-18000]],[13.84,0.2400000000000002,0.01764705882352943,2,2,3]],[[2024,2,29,15,16,null,null,[-18000]],[13.4,-0.1999999999999993,-0.014705882352941124,2,2,3]],[[2024,3,1,15,16,null,null,[-18000]],[13.11,-0.4900000000000002,-0.0360294117647059,2,2,3]],[[2024,3,4,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,3,5,15,16,null,null,[-18000]],[14.46,0.8600000000000012,0.06323529411764715,2,2,3]],[[2024,3,6,15,16,null,null,[-18000]],[14.5,0.9000000000000004,0.06617647058823532,2,2,3]],[[2024,3,7,15,16,null,null,[-18000]],[14.44,0.8399999999999999,0.06176470588235293,2,2,3]],[[2024,3,8,15,16,null,null,[-18000]],[14.74,1.1400000000000006,0.08382352941176476,2,2,3]],[[2024,3,11,15,16,null,null,[-18000]],[15.22,1.620000000000001,0.1191176470588236,2,2,2]],[[2024,3,12,15,16,null,null,[-18000]],[13.84,0.2400000000000002,0.01764705882352943,2,2,3]],[[2024,3,13,15,16,null,null,[-18000]],[13.75,0.15000000000000036,0.011029411764705909,2,2,3]],[[2024,3,14,15,16,null,null,[-18000]],[14.4,0.8000000000000007,0.05882352941176476,2,2,3]],[[2024,3,15,15,16,null,null,[-18000]],[14.41,0.8100000000000005,0.0595588235294118,2,2,3]],[[2024,3,18,15,16,null,null,[-18000]],[14.33,0.7300000000000004,0.053676470588235325,2,2,3]],[[2024,3,19,15,16,null,null,[-18000]],[13.82,0.22000000000000064,0.01617647058823534,2,2,3]],[[2024,3,20,15,16,null,null,[-18000]],[13.04,-0.5600000000000005,-0.041176470588235335,2,2,3]],[[2024,3,21,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,3,22,15,16,null,null,[-18000]],[13.06,-0.5399999999999991,-0.03970588235294111,2,2,3]],[[2024,3,25,15,16,null,null,[-18000]],[13.19,-0.41000000000000014,-0.030147058823529423,2,2,3]],[[2024,3,26,15,16,null,null,[-18000]],[13.24,-0.35999999999999943,-0.026470588235294076,2,2,3]],[[2024,3,27,15,16,null,null,[-18000]],[12.78,-0.8200000000000003,-0.060294117647058845,2,2,3]],[[2024,3,28,15,16,null,null,[-18000]],[13.01,-0.5899999999999999,-0.04338235294117646,2,2,3]],[[2024,4,1,15,16,null,null,[-18000]],[13.65,0.05000000000000071,0.0036764705882353465,2,3,4]],[[2024,4,2,15,16,null,null,[-18000]],[14.61,1.0099999999999998,0.07426470588235293,2,2,3]],[[2024,4,3,15,16,null,null,[-18000]],[14.33,0.7300000000000004,0.053676470588235325,2,2,3]],[[2024,4,4,15,16,null,null,[-18000]],[16.35,2.7500000000000018,0.20220588235294132,2,2,2]],[[2024,4,5,15,16,null,null,[-18000]],[16.03,2.4300000000000015,0.1786764705882354,2,2,2]],[[2024,4,8,15,16,null,null,[-18000]],[15.19,1.5899999999999999,0.11691176470588234,2,2,2]],[[2024,4,9,15,16,null,null,[-18000]],[14.98,1.3800000000000008,0.10147058823529417,2,2,2]],[[2024,4,10,15,16,null,null,[-18000]],[15.8,2.200000000000001,0.16176470588235303,2,2,2]],[[2024,4,11,15,16,null,null,[-18000]],[14.91,1.3100000000000005,0.09632352941176474,2,2,3]],[[2024,4,12,15,16,null,null,[-18000]],[17.31,3.709999999999999,0.27279411764705874,2,2,2]],[[2024,4,15,15,16,null,null,[-18000]],[19.23,5.630000000000001,0.4139705882352942,2,2,2]],[[2024,4,16,15,16,null,null,[-18000]],[18.4,4.799999999999999,0.35294117647058815,2,2,2]],[[2024,4,17,15,16,null,null,[-18000]],[18.21,4.610000000000001,0.3389705882352942,2,2,2]],[[2024,4,18,15,16,null,null,[-18000]],[18,4.4,0.3235294117647059,2,2,2]],[[2024,4,19,15,16,null,null,[-18000]],[18.71,5.110000000000001,0.37573529411764717,2,2,2]],[[2024,4,22,15,16,null,null,[-18000]],[16.94,3.3400000000000016,0.24558823529411777,2,2,2]],[[2024,4,23,15,16,null,null,[-18000]],[15.69,2.09,0.15367647058823528,2,2,2]],[[2024,4,24,15,16,null,null,[-18000]],[15.97,2.370000000000001,0.17426470588235302,2,2,2]],[[2024,4,25,15,16,null,null,[-18000]],[15.37,1.7699999999999996,0.1301470588235294,2,2,2]],[[2024,4,26,15,16,null,null,[-18000]],[15.03,1.4299999999999997,0.1051470588235294,2,2,2]],[[2024,4,29,15,16,null,null,[-18000]],[14.67,1.0700000000000003,0.07867647058823532,2,2,3]],[[2024,4,30,15,16,null,null,[-18000]],[15.65,2.0500000000000007,0.1507352941176471,2,2,2]],[[2024,5,1,15,16,null,null,[-18000]],[15.39,1.790000000000001,0.1316176470588236,2,2,2]],[[2024,5,2,15,16,null,null,[-18000]],[14.68,1.08,0.07941176470588236,2,2,3]],[[2024,5,3,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,5,6,15,16,null,null,[-18000]],[13.49,-0.10999999999999943,-0.008088235294117606,2,2,4]],[[2024,5,7,15,16,null,null,[-18000]],[13.23,-0.3699999999999992,-0.027205882352941118,2,2,3]],[[2024,5,8,15,16,null,null,[-18000]],[13,-0.5999999999999996,-0.044117647058823505,2,2,3]],[[2024,5,9,15,16,null,null,[-18000]],[12.69,-0.9100000000000001,-0.06691176470588237,2,2,3]],[[2024,5,10,15,16,null,null,[-18000]],[12.55,-1.049999999999999,-0.0772058823529411,2,2,3]],[[2024,5,13,15,16,null,null,[-18000]],[13.6,0,0,2,2,2]],[[2024,5,14,15,16,null,null,[-18000]],[13.42,-0.17999999999999972,-0.013235294117647038,2,2,3]],[[2024,5,15,15,16,null,null,[-18000]],[12.45,-1.1500000000000004,-0.0845588235294118,2,2,3]],[[2024,5,16,15,16,null,null,[-18000]],[12.42,-1.1799999999999997,-0.08676470588235292,2,2,3]],[[2024,5,17,15,16,null,null,[-18000]],[11.99,-1.6099999999999994,-0.11838235294117644,2,2,2]],[[2024,5,20,15,16,null,null,[-18000]],[12.15,-1.4499999999999993,-0.10661764705882348,2,2,2]],[[2024,5,21,15,16,null,null,[-18000]],[11.86,-1.7400000000000002,-0.12794117647058825,2,2,2]],[[2024,5,22,15,16,null,null,[-18000]],[12.29,-1.3100000000000005,-0.09632352941176474,2,2,3]],[[2024,5,23,15,16,null,null,[-18000]],[12.77,-0.8300000000000001,-0.06102941176470589,2,2,3]],[[2024,5,24,15,16,null,null,[-18000]],[11.93,-1.67,-0.12279411764705882,2,2,2]],[[2024,5,28,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,5,29,15,16,null,null,[-18000]],[14.28,0.6799999999999997,0.04999999999999998,2,2,3]],[[2024,5,30,15,16,null,null,[-18000]],[14.47,0.870000000000001,0.0639705882352942,2,2,3]],[[2024,5,31,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,6,3,15,16,null,null,[-18000]],[13.11,-0.4900000000000002,-0.0360294117647059,2,2,3]],[[2024,6,4,15,16,null,null,[-18000]],[13.16,-0.4399999999999995,-0.03235294117647055,2,2,3]],[[2024,6,5,15,16,null,null,[-18000]],[12.63,-0.9699999999999989,-0.07132352941176462,2,2,3]],[[2024,6,6,15,16,null,null,[-18000]],[12.58,-1.0199999999999996,-0.07499999999999997,2,2,3]],[[2024,6,7,15,16,null,null,[-18000]],[12.22,-1.379999999999999,-0.10147058823529405,2,2,2]],[[2024,6,10,15,16,null,null,[-18000]],[12.74,-0.8599999999999994,-0.06323529411764701,2,2,3]],[[2024,6,11,15,16,null,null,[-18000]],[12.85,-0.75,-0.05514705882352941,2,2,3]],[[2024,6,12,15,16,null,null,[-18000]],[12.04,-1.5600000000000005,-0.11470588235294121,2,2,2]],[[2024,6,13,15,16,null,null,[-18000]],[11.94,-1.6600000000000001,-0.12205882352941178,2,2,2]],[[2024,6,14,15,16,null,null,[-18000]],[12.66,-0.9399999999999995,-0.06911764705882349,2,2,3]],[[2024,6,17,15,16,null,null,[-18000]],[12.75,-0.8499999999999996,-0.06249999999999997,2,2,3]],[[2024,6,18,15,16,null,null,[-18000]],[12.3,-1.299999999999999,-0.09558823529411757,2,2,3]],[[2024,6,20,15,16,null,null,[-18000]],[13.28,-0.3200000000000003,-0.023529411764705903,2,2,3]],[[2024,6,21,15,16,null,null,[-18000]],[13.2,-0.40000000000000036,-0.02941176470588238,2,2,3]],[[2024,6,24,15,16,null,null,[-18000]],[13.33,-0.2699999999999996,-0.019852941176470556,2,2,3]],[[2024,6,25,15,16,null,null,[-18000]],[12.84,-0.7599999999999998,-0.05588235294117646,2,2,3]],[[2024,6,26,15,16,null,null,[-18000]],[12.55,-1.049999999999999,-0.0772058823529411,2,2,3]],[[2024,6,27,15,16,null,null,[-18000]],[12.24,-1.3599999999999994,-0.09999999999999996,2,2,3]],[[2024,6,28,15,16,null,null,[-18000]],[12.44,-1.1600000000000001,-0.08529411764705884,2,2,3]],[[2024,7,1,15,16,null,null,[-18000]],[12.22,-1.379999999999999,-0.10147058823529405,2,2,2]],[[2024,7,2,15,16,null,null,[-18000]],[12.03,-1.5700000000000003,-0.11544117647058826,2,2,2]],[[2024,7,3,13,null,null,null,[-18000]],[12.09,-1.5099999999999998,-0.11102941176470588,2,2,2]],[[2024,7,5,15,16,null,null,[-18000]],[12.48,-1.1199999999999992,-0.08235294117647053,2,2,3]],[[2024,7,8,15,16,null,null,[-18000]],[12.37,-1.2300000000000004,-0.09044117647058827,2,2,3]],[[2024,7,9,15,16,null,null,[-18000]],[12.51,-1.0899999999999999,-0.0801470588235294,2,2,3]],[[2024,7,10,15,16,null,null,[-18000]],[12.85,-0.75,-0.05514705882352941,2,2,3]],[[2024,7,11,15,16,null,null,[-18000]],[12.92,-0.6799999999999997,-0.04999999999999998,2,2,3]],[[2024,7,12,15,16,null,null,[-18000]],[12.46,-1.1399999999999988,-0.08382352941176462,2,2,3]],[[2024,7,15,15,16,null,null,[-18000]],[13.12,-0.4800000000000004,-0.03529411764705886,2,2,3]],[[2024,7,16,15,16,null,null,[-18000]],[13.19,-0.41000000000000014,-0.030147058823529423,2,2,3]],[[2024,7,17,15,16,null,null,[-18000]],[14.48,0.8800000000000008,0.06470588235294124,2,2,3]],[[2024,7,18,15,16,null,null,[-18000]],[15.93,2.33,0.1713235294117647,2,2,2]],[[2024,7,19,15,16,null,null,[-18000]],[16.52,2.92,0.21470588235294116,2,2,2]],[[2024,7,22,15,16,null,null,[-18000]],[14.91,1.3100000000000005,0.09632352941176474,2,2,3]],[[2024,7,23,15,16,null,null,[-18000]],[14.72,1.120000000000001,0.08235294117647067,2,2,3]],[[2024,7,24,15,16,null,null,[-18000]],[18.04,4.4399999999999995,0.32647058823529407,2,2,2]],[[2024,7,25,15,16,null,null,[-18000]],[18.46,4.860000000000001,0.3573529411764707,2,2,2]],[[2024,7,26,15,16,null,null,[-18000]],[16.39,2.790000000000001,0.2051470588235295,2,2,2]]]]],null,-18000,18.46,\"VIX指数\",86400,1],[[\"RUT\",\"INDEXRUSSELL\"],\"/m/04zvfw\",null,[[[1],[[[2024,1,29,16,null,null,null,[-14400]],[2011.42,0,0,2,2,2]],[[2024,1,30,16,null,null,null,[-14400]],[1996.24,-15.180000000000064,-0.007546907160115771,2,2,4]],[[2024,1,31,16,null,null,null,[-14400]],[1947.34,-64.08000000000015,-0.031858090304362166,2,2,3]],[[2024,2,1,16,null,null,null,[-14400]],[1974.42,-37,-0.018394964751270247,2,2,3]],[[2024,2,2,16,null,null,null,[-14400]],[1962.73,-48.690000000000055,-0.024206779290252684,2,2,3]],[[2024,2,5,16,null,null,null,[-14400]],[1937.24,-74.18000000000006,-0.0368794185202494,2,2,3]],[[2024,2,6,16,null,null,null,[-14400]],[1953.63,-57.789999999999964,-0.02873094629664613,2,2,3]],[[2024,2,7,16,null,null,null,[-14400]],[1950.36,-61.06000000000017,-0.03035666345169093,2,2,3]],[[2024,2,8,16,null,null,null,[-14400]],[1979.7,-31.720000000000027,-0.015769953565143047,2,2,3]],[[2024,2,9,16,null,null,null,[-14400]],[2009.99,-1.4300000000000637,-7.10940529576152E-4,2,2,5]],[[2024,2,12,16,null,null,null,[-14400]],[2045.26,33.83999999999992,0.01682393532926983,2,2,3]],[[2024,2,13,16,null,null,null,[-14400]],[1964.17,-47.25,-0.023490867148581597,2,2,3]],[[2024,2,14,16,null,null,null,[-14400]],[2012.1,0.6799999999998363,3.380696224556961E-4,2,2,5]],[[2024,2,15,16,null,null,null,[-14400]],[2061.48,50.059999999999945,0.02488789014725912,2,2,3]],[[2024,2,16,16,null,null,null,[-14400]],[2032.74,21.319999999999936,0.01059947698640758,2,2,3]],[[2024,2,20,16,null,null,null,[-14400]],[2004.14,-7.279999999999973,-0.003619333605114781,2,2,4]],[[2024,2,21,16,null,null,null,[-14400]],[1994.74,-16.680000000000064,-0.008292648974356456,2,2,4]],[[2024,2,22,16,null,null,null,[-14400]],[2013.84,2.4199999999998454,0.001203130126974896,2,2,4]],[[2024,2,23,16,null,null,null,[-14400]],[2016.69,5.269999999999982,0.0026200395740322664,2,2,4]],[[2024,2,26,16,null,null,null,[-14400]],[2028.97,17.549999999999955,0.008725179226616,2,2,4]],[[2024,2,27,16,null,null,null,[-14400]],[2056.11,44.690000000000055,0.02221813445227752,2,2,3]],[[2024,2,28,16,null,null,null,[-14400]],[2040.31,28.889999999999873,0.014362987342275542,2,2,3]],[[2024,2,29,16,null,null,null,[-14400]],[2054.84,43.42000000000007,0.021586739716220417,2,2,3]],[[2024,3,1,16,null,null,null,[-14400]],[2076.39,64.9699999999998,0.03230056378081147,2,2,3]],[[2024,3,4,16,null,null,null,[-14400]],[2074.31,62.88999999999987,0.031266468465064416,2,2,3]],[[2024,3,5,16,null,null,null,[-14400]],[2053.71,42.289999999999964,0.02102494754949238,2,2,3]],[[2024,3,6,16,null,null,null,[-14400]],[2068.09,56.67000000000007,0.02817412574201314,2,2,3]],[[2024,3,7,16,null,null,null,[-14400]],[2084.74,73.31999999999971,0.03645185988008457,2,2,3]],[[2024,3,8,16,null,null,null,[-14400]],[2082.71,71.28999999999996,0.0354426226248123,2,2,3]],[[2024,3,11,16,null,null,null,[-14400]],[2065.88,54.460000000000036,0.027075399469031845,2,2,3]],[[2024,3,12,16,null,null,null,[-14400]],[2065.48,54.059999999999945,0.026876534985234284,2,2,3]],[[2024,3,13,16,null,null,null,[-14400]],[2071.71,60.289999999999964,0.029973849320380608,2,2,3]],[[2024,3,14,16,null,null,null,[-14400]],[2031.18,19.75999999999999,0.009823905499597295,2,2,4]],[[2024,3,15,16,null,null,null,[-14400]],[2039.32,27.899999999999864,0.013870797744876685,2,2,3]],[[2024,3,18,16,null,null,null,[-14400]],[2024.74,13.319999999999936,0.006622187310457257,2,2,4]],[[2024,3,19,16,null,null,null,[-14400]],[2035.71,24.289999999999964,0.012076045778604152,2,2,3]],[[2024,3,20,16,null,null,null,[-14400]],[2074.88,63.460000000000036,0.03154985035447596,2,2,3]],[[2024,3,21,16,null,null,null,[-14400]],[2098.56,87.13999999999987,0.04332262779528884,2,2,3]],[[2024,3,22,16,null,null,null,[-14400]],[2072,60.57999999999993,0.03011802607113379,2,2,3]],[[2024,3,25,16,null,null,null,[-14400]],[2074.16,62.73999999999978,0.031191894283640303,2,2,3]],[[2024,3,26,16,null,null,null,[-14400]],[2070.16,58.73999999999978,0.02920324944566514,2,2,3]],[[2024,3,27,16,null,null,null,[-14400]],[2114.35,102.92999999999984,0.05117280329319577,2,2,3]],[[2024,3,28,16,null,null,null,[-14400]],[2124.55,113.13000000000011,0.05624384763003257,2,2,3]],[[2024,4,1,16,null,null,null,[-14400]],[2102.84,91.42000000000007,0.045450477771922355,2,2,3]],[[2024,4,2,16,null,null,null,[-14400]],[2065.04,53.61999999999989,0.02665778405305699,2,2,3]],[[2024,4,3,16,null,null,null,[-14400]],[2076.2,64.77999999999975,0.032206103151007616,2,2,3]],[[2024,4,4,16,null,null,null,[-14400]],[2053.83,42.409999999999854,0.02108460689463158,2,2,3]],[[2024,4,5,16,null,null,null,[-14400]],[2063.47,52.04999999999973,0.025877240954151656,2,2,3]],[[2024,4,8,16,null,null,null,[-14400]],[2073.71,62.289999999999964,0.030968171739368187,2,2,3]],[[2024,4,9,16,null,null,null,[-14400]],[2080.8,69.38000000000011,0.03449304471467923,2,2,3]],[[2024,4,10,16,null,null,null,[-14400]],[2028.39,16.970000000000027,0.008436825725109637,2,2,4]],[[2024,4,11,16,null,null,null,[-14400]],[2042.6,31.179999999999836,0.015501486512016304,2,2,3]],[[2024,4,12,16,null,null,null,[-14400]],[2003.17,-8.25,-0.004101579978323771,2,2,4]],[[2024,4,15,16,null,null,null,[-14400]],[1975.71,-35.710000000000036,-0.017753626791023276,2,2,3]],[[2024,4,16,16,null,null,null,[-14400]],[1967.48,-43.940000000000055,-0.02184526354515718,2,2,3]],[[2024,4,17,16,null,null,null,[-14400]],[1947.95,-63.47000000000003,-0.03155482196657089,2,2,3]],[[2024,4,18,16,null,null,null,[-14400]],[1942.96,-68.46000000000004,-0.03403565640194491,2,2,3]],[[2024,4,19,16,null,null,null,[-14400]],[1947.66,-63.75999999999999,-0.031698998717324074,2,2,3]],[[2024,4,22,16,null,null,null,[-14400]],[1967.47,-43.950000000000045,-0.021850235157252113,2,2,3]],[[2024,4,23,16,null,null,null,[-14400]],[2002.64,-8.779999999999973,-0.0043650754193554665,2,2,4]],[[2024,4,24,16,null,null,null,[-14400]],[1995.43,-15.990000000000009,-0.007949607739805714,2,2,4]],[[2024,4,25,16,null,null,null,[-14400]],[1981.12,-30.300000000000182,-0.01506398464766194,2,2,3]],[[2024,4,26,16,null,null,null,[-14400]],[2002,-9.420000000000073,-0.004683258593431542,2,2,4]],[[2024,4,29,16,null,null,null,[-14400]],[2016.03,4.6099999999999,0.0022919131757663243,2,2,4]],[[2024,4,30,16,null,null,null,[-14400]],[1973.91,-37.50999999999999,-0.018648516968112074,2,2,3]],[[2024,5,1,16,null,null,null,[-14400]],[1980.23,-31.190000000000055,-0.01550645812411135,2,2,3]],[[2024,5,2,16,null,null,null,[-14400]],[2016.11,4.689999999999827,0.0023316860725257913,2,2,4]],[[2024,5,3,16,null,null,null,[-14400]],[2035.72,24.299999999999955,0.012081017390699085,2,2,3]],[[2024,5,6,16,null,null,null,[-14400]],[2060.67,49.25,0.02448518956756918,2,2,3]],[[2024,5,7,16,null,null,null,[-14400]],[2064.65,53.23000000000002,0.026463891181354473,2,2,3]],[[2024,5,8,16,null,null,null,[-14400]],[2055.14,43.7199999999998,0.021735888079068417,2,2,3]],[[2024,5,9,16,null,null,null,[-14400]],[2073.63,62.210000000000036,0.03092839884260872,2,2,3]],[[2024,5,10,16,null,null,null,[-14400]],[2059.78,48.36000000000013,0.02404271609111977,2,2,3]],[[2024,5,13,16,null,null,null,[-14400]],[2062.12,50.69999999999982,0.025206073321335086,2,2,3]],[[2024,5,14,16,null,null,null,[-14400]],[2085.69,74.26999999999998,0.036924163029103806,2,2,3]],[[2024,5,15,16,null,null,null,[-14400]],[2109.46,98.03999999999996,0.048741684978771196,2,2,3]],[[2024,5,16,16,null,null,null,[-14400]],[2096.25,84.82999999999993,0.04217418540135821,2,2,3]],[[2024,5,17,16,null,null,null,[-14400]],[2095.72,84.29999999999973,0.041910689960326396,2,2,3]],[[2024,5,20,16,null,null,null,[-14400]],[2102.5,91.07999999999993,0.0452814429606944,2,2,3]],[[2024,5,21,16,null,null,null,[-14400]],[2098.36,86.94000000000005,0.043223195553390166,2,2,3]],[[2024,5,22,16,null,null,null,[-14400]],[2081.71,70.28999999999996,0.03494546141531851,2,2,3]],[[2024,5,23,16,null,null,null,[-14400]],[2048.41,36.98999999999978,0.0183899931391752,2,2,3]],[[2024,5,24,16,null,null,null,[-14400]],[2069.67,58.25,0.028959640453013292,2,2,3]],[[2024,5,28,16,null,null,null,[-14400]],[2066.85,55.429999999999836,0.027557645842240724,2,2,3]],[[2024,5,29,16,null,null,null,[-14400]],[2036.19,24.769999999999982,0.01231468315916118,2,2,3]],[[2024,5,30,16,null,null,null,[-14400]],[2056.6,45.179999999999836,0.02246174344492937,2,2,3]],[[2024,5,31,16,null,null,null,[-14400]],[2070.13,58.710000000000036,0.029188334609380456,2,2,3]],[[2024,6,3,16,null,null,null,[-14400]],[2059.68,48.25999999999976,0.02399299997017021,2,2,3]],[[2024,6,4,16,null,null,null,[-14400]],[2033.94,22.519999999999982,0.011196070437800151,2,2,3]],[[2024,6,5,16,null,null,null,[-14400]],[2063.87,52.44999999999982,0.026076105437949217,2,2,3]],[[2024,6,6,16,null,null,null,[-14400]],[2049.44,38.01999999999998,0.018902069184953903,2,2,3]],[[2024,6,7,16,null,null,null,[-14400]],[2026.55,15.129999999999882,0.00752204909964099,2,2,4]],[[2024,6,10,16,null,null,null,[-14400]],[2031.61,20.189999999999827,0.010037684819679543,2,2,3]],[[2024,6,11,16,null,null,null,[-14400]],[2024.35,12.929999999999836,0.006428294438754629,2,2,4]],[[2024,6,12,16,null,null,null,[-14400]],[2057.1,45.679999999999836,0.022710324049676267,2,2,3]],[[2024,6,13,16,null,null,null,[-14400]],[2038.91,27.49000000000001,0.013666961648984304,2,2,3]],[[2024,6,14,16,null,null,null,[-14400]],[2006.16,-5.259999999999991,-0.002615067961937333,2,2,4]],[[2024,6,17,16,null,null,null,[-14400]],[2022.01,10.589999999999918,0.0052649372085392,2,2,4]],[[2024,6,18,16,null,null,null,[-14400]],[2025.23,13.809999999999945,0.006865796303109219,2,2,4]],[[2024,6,20,16,null,null,null,[-14400]],[2017.39,5.970000000000027,0.0029680524206779427,2,2,4]],[[2024,6,21,16,null,null,null,[-14400]],[2022.03,10.6099999999999,0.005274880432729067,2,2,4]],[[2024,6,24,16,null,null,null,[-14400]],[2030.81,19.389999999999873,0.009639955852084534,2,2,4]],[[2024,6,25,16,null,null,null,[-14400]],[2022.35,10.929999999999836,0.005433972019767048,2,2,4]],[[2024,6,26,16,null,null,null,[-14400]],[2018.12,6.699999999999818,0.0033309801036083054,2,2,4]],[[2024,6,27,16,null,null,null,[-14400]],[2038.34,26.919999999999845,0.013383579759572762,2,2,3]],[[2024,6,28,16,null,null,null,[-14400]],[2047.69,36.26999999999998,0.01803203706833977,2,2,3]],[[2024,7,1,16,null,null,null,[-14400]],[2030.07,18.649999999999864,0.009272056557059124,2,2,4]],[[2024,7,2,16,null,null,null,[-14400]],[2033.87,22.449999999999818,0.011161269153135506,2,2,3]],[[2024,7,3,13,5,null,null,[-14400]],[2036.62,25.199999999999818,0.012528462479243429,2,2,3]],[[2024,7,5,16,null,null,null,[-14400]],[2026.73,15.309999999999945,0.007611538117349904,2,2,4]],[[2024,7,8,16,null,null,null,[-14400]],[2038.67,27.25,0.013547642958705789,2,2,3]],[[2024,7,9,16,null,null,null,[-14400]],[2029.47,18.049999999999955,0.008973759831362895,2,2,4]],[[2024,7,10,16,null,null,null,[-14400]],[2051.75,40.32999999999993,0.02005051157888453,2,2,3]],[[2024,7,11,16,null,null,null,[-14400]],[2125.04,113.61999999999989,0.056487456622684416,2,2,3]],[[2024,7,12,16,null,null,null,[-14400]],[2148.27,136.8499999999999,0.06803651151922517,2,2,3]],[[2024,7,15,16,null,null,null,[-14400]],[2187.02,175.5999999999999,0.08730150838710955,2,2,3]],[[2024,7,16,16,null,null,null,[-14400]],[2263.67,252.25,0.12540891509480864,2,2,2]],[[2024,7,17,16,null,null,null,[-14400]],[2239.67,228.25,0.11347704606695767,2,2,2]],[[2024,7,18,16,null,null,null,[-14400]],[2198.29,186.8699999999999,0.09290451521810457,2,2,3]],[[2024,7,19,16,null,null,null,[-14400]],[2184.35,172.92999999999984,0.0859740879577611,2,2,3]],[[2024,7,22,16,null,null,null,[-14400]],[2220.65,209.23000000000002,0.10402103986238578,2,2,2]],[[2024,7,23,16,null,null,null,[-14400]],[2243.27,231.8499999999999,0.11526682642113527,2,2,2]],[[2024,7,24,16,null,null,null,[-14400]],[2195.37,183.94999999999982,0.09145280448638266,2,2,3]],[[2024,7,25,16,null,null,null,[-14400]],[2222.98,211.55999999999995,0.10517942548050628,2,2,2]],[[2024,7,26,16,null,null,null,[-14400]],[2260.07,248.6500000000001,0.12361913474063103,2,2,2]],[[2024,7,26,16,30,8,null,[-14400]],[2260.0688,248.64879999999994,0.12361853814717957,2,2,2]]]]],null,-14400,2222.9805,\"ラッセル2000インデックス\",86400,1],[[\"NI225\",\"INDEXNIKKEI\"],\"/m/0b18t\",null,[[[1],[[[2024,1,29,15,null,null,null,[32400]],[36026.94,0,0,2,2,2]],[[2024,1,30,15,null,null,null,[32400]],[36065.86,38.919999999998254,0.0010803026846020853,2,2,4]],[[2024,1,31,15,null,null,null,[32400]],[36286.71,259.7699999999968,0.007210437522587174,2,2,4]],[[2024,2,1,15,null,null,null,[32400]],[36011.46,-15.480000000003201,-4.2967845728788515E-4,2,2,5]],[[2024,2,2,15,null,null,null,[32400]],[36158.02,131.07999999999447,0.003638388383803744,2,2,4]],[[2024,2,5,15,null,null,null,[32400]],[36354.16,327.22000000000116,0.0090826475964931,2,2,4]],[[2024,2,6,15,null,null,null,[32400]],[36160.66,133.72000000000116,0.003711666880395647,2,2,4]],[[2024,2,7,15,null,null,null,[32400]],[36119.92,92.97999999999593,0.0025808464443551385,2,2,4]],[[2024,2,8,15,null,null,null,[32400]],[36863.28,836.3399999999965,0.023214294636180494,2,2,3]],[[2024,2,9,15,null,null,null,[32400]],[36897.42,870.4799999999959,0.024161918830741546,2,2,3]],[[2024,2,13,15,null,null,null,[32400]],[37963.97,1937.0299999999988,0.05376615388373253,2,2,3]],[[2024,2,14,15,null,null,null,[32400]],[37703.32,1676.3799999999974,0.04653129019561465,2,2,3]],[[2024,2,15,15,null,null,null,[32400]],[38157.94,2131,0.059150180392783844,2,2,3]],[[2024,2,16,15,null,null,null,[32400]],[38487.24,2460.2999999999956,0.06829056256234906,2,2,3]],[[2024,2,19,15,null,null,null,[32400]],[38470.38,2443.439999999995,0.0678225794363883,2,2,3]],[[2024,2,20,15,null,null,null,[32400]],[38363.61,2336.6699999999983,0.06485896387536655,2,2,3]],[[2024,2,21,15,null,null,null,[32400]],[38262.16,2235.220000000001,0.06204301558777962,2,2,3]],[[2024,2,22,15,null,null,null,[32400]],[39098.68,3071.739999999998,0.08526230648509137,2,2,3]],[[2024,2,26,15,null,null,null,[32400]],[39233.71,3206.769999999997,0.08901033504371997,2,2,3]],[[2024,2,27,15,null,null,null,[32400]],[39239.52,3212.5799999999945,0.08917160325023425,2,2,3]],[[2024,2,28,15,null,null,null,[32400]],[39208.03,3181.0899999999965,0.0882975351223278,2,2,3]],[[2024,2,29,15,null,null,null,[32400]],[39166.19,3139.25,0.08713618197937432,2,2,3]],[[2024,3,1,15,null,null,null,[32400]],[39910.82,3883.8799999999974,0.10780488156918121,2,2,2]],[[2024,3,4,15,null,null,null,[32400]],[40109.23,4082.290000000001,0.11331214918613683,2,2,2]],[[2024,3,5,15,null,null,null,[32400]],[40097.63,4070.689999999995,0.11299016791323367,2,2,2]],[[2024,3,6,15,null,null,null,[32400]],[40090.78,4063.8399999999965,0.11280003242018323,2,2,2]],[[2024,3,7,15,null,null,null,[32400]],[39598.71,3571.769999999997,0.09914164233765056,2,2,3]],[[2024,3,8,15,null,null,null,[32400]],[39688.94,3662,0.10164615701472286,2,2,2]],[[2024,3,11,15,null,null,null,[32400]],[38820.49,2793.5499999999956,0.07754058490673911,2,2,3]],[[2024,3,12,15,null,null,null,[32400]],[38797.51,2770.5699999999997,0.07690272890231586,2,2,3]],[[2024,3,13,15,null,null,null,[32400]],[38695.97,2669.029999999999,0.0740842824841632,2,2,3]],[[2024,3,14,15,null,null,null,[32400]],[38807.38,2780.439999999995,0.07717669055434613,2,2,3]],[[2024,3,15,15,null,null,null,[32400]],[38707.64,2680.699999999997,0.07440820674750609,2,2,3]],[[2024,3,18,15,null,null,null,[32400]],[39740.44,3713.5,0.10307564283838705,2,2,2]],[[2024,3,19,15,null,null,null,[32400]],[40003.6,3976.659999999996,0.11038017661227947,2,2,2]],[[2024,3,21,15,null,null,null,[32400]],[40815.66,4788.720000000001,0.1329205311358667,2,2,2]],[[2024,3,22,15,null,null,null,[32400]],[40888.43,4861.489999999998,0.13494040848320724,2,2,2]],[[2024,3,25,15,null,null,null,[32400]],[40414.12,4387.18,0.12177498283229161,2,2,2]],[[2024,3,26,15,null,null,null,[32400]],[40398.03,4371.0899999999965,0.12132837260117002,2,2,2]],[[2024,3,27,15,null,null,null,[32400]],[40762.73,4735.790000000001,0.13145135279321532,2,2,2]],[[2024,3,28,15,null,null,null,[32400]],[40168.07,4141.129999999997,0.11494537143593092,2,2,2]],[[2024,3,29,15,null,null,null,[32400]],[40369.44,4342.5,0.12053479979148936,2,2,2]],[[2024,4,1,15,null,null,null,[32400]],[39803.09,3776.149999999994,0.10481461928212593,2,2,2]],[[2024,4,2,15,null,null,null,[32400]],[39838.91,3811.970000000001,0.10580887524724555,2,2,2]],[[2024,4,3,15,null,null,null,[32400]],[39451.85,3424.909999999996,0.09506524839467342,2,2,3]],[[2024,4,4,15,null,null,null,[32400]],[39773.14,3746.199999999997,0.103983296943898,2,2,2]],[[2024,4,5,15,null,null,null,[32400]],[38992.08,2965.1399999999994,0.08230340961513799,2,2,3]],[[2024,4,8,15,null,null,null,[32400]],[39347.04,3320.0999999999985,0.09215603656596975,2,2,3]],[[2024,4,9,15,null,null,null,[32400]],[39773.13,3746.189999999995,0.1039830193738351,2,2,2]],[[2024,4,10,15,null,null,null,[32400]],[39581.81,3554.8699999999953,0.0986725489314384,2,2,3]],[[2024,4,11,15,null,null,null,[32400]],[39442.63,3415.689999999995,0.09480932879672808,2,2,3]],[[2024,4,12,15,null,null,null,[32400]],[39523.55,3496.6100000000006,0.09705542574528951,2,2,3]],[[2024,4,15,15,null,null,null,[32400]],[39232.8,3205.8600000000006,0.08898507616800096,2,2,3]],[[2024,4,16,15,null,null,null,[32400]],[38471.2,2444.2599999999948,0.06784534018154177,2,2,3]],[[2024,4,17,15,null,null,null,[32400]],[37961.8,1934.8600000000006,0.053705921180094686,2,2,3]],[[2024,4,18,15,null,null,null,[32400]],[38079.7,2052.7599999999948,0.05697847222106553,2,2,3]],[[2024,4,19,15,null,null,null,[32400]],[37068.35,1041.4099999999962,0.028906423914992395,2,2,3]],[[2024,4,22,15,null,null,null,[32400]],[37438.61,1411.6699999999983,0.03918373306198079,2,2,3]],[[2024,4,23,15,null,null,null,[32400]],[37552.16,1525.2200000000012,0.04233554112561325,2,2,3]],[[2024,4,24,15,null,null,null,[32400]],[38460.08,2433.1399999999994,0.06753668227165557,2,2,3]],[[2024,4,25,15,null,null,null,[32400]],[37628.48,1601.5400000000009,0.0444539558452647,2,2,3]],[[2024,4,26,15,null,null,null,[32400]],[37934.76,1907.8199999999997,0.05295537173015526,2,2,3]],[[2024,4,30,15,null,null,null,[32400]],[38405.66,2378.720000000001,0.06602614598964,2,2,3]],[[2024,5,1,15,null,null,null,[32400]],[38274.05,2247.1100000000006,0.062373046392505176,2,2,3]],[[2024,5,2,15,null,null,null,[32400]],[38236.07,2209.1299999999974,0.06131883529381061,2,2,3]],[[2024,5,7,15,null,null,null,[32400]],[38835.1,2808.159999999996,0.0779461147685592,2,2,3]],[[2024,5,8,15,null,null,null,[32400]],[38202.37,2175.4300000000003,0.06038342418201491,2,2,3]],[[2024,5,9,15,null,null,null,[32400]],[38073.98,2047.0400000000009,0.05681970214511698,2,2,3]],[[2024,5,10,15,null,null,null,[32400]],[38229.11,2202.1699999999983,0.061125646530068835,2,2,3]],[[2024,5,13,15,null,null,null,[32400]],[38179.46,2152.519999999997,0.059747511168031384,2,2,3]],[[2024,5,14,15,null,null,null,[32400]],[38356.06,2329.1199999999953,0.06464939847791668,2,2,3]],[[2024,5,15,15,null,null,null,[32400]],[38385.73,2358.790000000001,0.0654729488543851,2,2,3]],[[2024,5,16,15,null,null,null,[32400]],[38920.26,2893.3199999999997,0.08030990142376787,2,2,3]],[[2024,5,17,15,null,null,null,[32400]],[38787.38,2760.439999999995,0.0766215504286513,2,2,3]],[[2024,5,20,15,null,null,null,[32400]],[39069.68,3042.739999999998,0.08445735330283387,2,2,3]],[[2024,5,21,15,null,null,null,[32400]],[38946.93,2919.989999999998,0.08105018078138188,2,2,3]],[[2024,5,22,15,null,null,null,[32400]],[38617.1,2590.159999999996,0.07189508739848557,2,2,3]],[[2024,5,23,15,null,null,null,[32400]],[39103.22,3076.279999999999,0.08538832329362413,2,2,3]],[[2024,5,24,15,null,null,null,[32400]],[38646.11,2619.1699999999983,0.07270031815080598,2,2,3]],[[2024,5,27,15,null,null,null,[32400]],[38900.02,2873.0799999999945,0.07974809961656455,2,2,3]],[[2024,5,28,15,null,null,null,[32400]],[38855.37,2828.4300000000003,0.07850874928595102,2,2,3]],[[2024,5,29,15,null,null,null,[32400]],[38556.87,2529.9300000000003,0.07022328290995572,2,2,3]],[[2024,5,30,15,null,null,null,[32400]],[38054.13,2027.189999999995,0.056268725570364705,2,2,3]],[[2024,5,31,15,null,null,null,[32400]],[38487.9,2460.959999999999,0.06830888218649707,2,2,3]],[[2024,6,3,15,null,null,null,[32400]],[38923.03,2896.0899999999965,0.08038678833117652,2,2,3]],[[2024,6,4,15,null,null,null,[32400]],[38837.46,2810.519999999997,0.0780116213033912,2,2,3]],[[2024,6,5,15,null,null,null,[32400]],[38490.17,2463.229999999996,0.06837189059076335,2,2,3]],[[2024,6,6,15,null,null,null,[32400]],[38703.51,2676.5699999999997,0.07429357031155018,2,2,3]],[[2024,6,7,15,null,null,null,[32400]],[38683.93,2656.989999999998,0.07375008812849489,2,2,3]],[[2024,6,10,15,null,null,null,[32400]],[39038.16,3011.220000000001,0.08358245246473892,2,2,3]],[[2024,6,11,15,null,null,null,[32400]],[39134.79,3107.8499999999985,0.0862646119820334,2,2,3]],[[2024,6,12,15,null,null,null,[32400]],[38876.71,2849.769999999997,0.0791010838000673,2,2,3]],[[2024,6,13,15,null,null,null,[32400]],[38720.47,2693.529999999999,0.07476432913813937,2,2,3]],[[2024,6,14,15,null,null,null,[32400]],[38814.56,2787.6199999999953,0.07737598585947059,2,2,3]],[[2024,6,17,15,null,null,null,[32400]],[38102.44,2075.5,0.0576096665439807,2,2,3]],[[2024,6,18,15,null,null,null,[32400]],[38482.11,2455.1699999999983,0.0681481691201084,2,2,3]],[[2024,6,19,15,null,null,null,[32400]],[38570.76,2543.8199999999997,0.07060882772725076,2,2,3]],[[2024,6,20,15,null,null,null,[32400]],[38633.02,2606.0799999999945,0.07233697893853862,2,2,3]],[[2024,6,21,15,null,null,null,[32400]],[38596.47,2569.529999999999,0.07132246035883144,2,2,3]],[[2024,6,24,15,null,null,null,[32400]],[38804.65,2777.709999999999,0.0771009139271889,2,2,3]],[[2024,6,25,15,null,null,null,[32400]],[39173.15,3146.209999999999,0.0873293707431161,2,2,3]],[[2024,6,26,15,null,null,null,[32400]],[39667.07,3640.1299999999974,0.1010391112872755,2,2,2]],[[2024,6,27,15,null,null,null,[32400]],[39341.54,3314.5999999999985,0.09200337303140367,2,2,3]],[[2024,6,28,15,null,null,null,[32400]],[39583.08,3556.1399999999994,0.09870780032942013,2,2,3]],[[2024,7,1,15,null,null,null,[32400]],[39631.06,3604.1199999999953,0.1000395814909619,2,2,2]],[[2024,7,2,15,null,null,null,[32400]],[40074.69,4047.75,0.11235342218906184,2,2,2]],[[2024,7,3,15,null,null,null,[32400]],[40580.76,4553.82,0.12640041035958088,2,2,2]],[[2024,7,4,15,null,null,null,[32400]],[40913.65,4886.709999999999,0.13564044018170843,2,2,2]],[[2024,7,5,15,null,null,null,[32400]],[40912.37,4885.43,0.135604911213664,2,2,2]],[[2024,7,8,15,null,null,null,[32400]],[40780.7,4753.759999999995,0.13195014619615195,2,2,2]],[[2024,7,9,15,null,null,null,[32400]],[41580.17,5553.229999999996,0.15414104001061416,2,2,2]],[[2024,7,10,15,null,null,null,[32400]],[41831.99,5805.049999999996,0.16113080933323773,2,2,2]],[[2024,7,11,15,null,null,null,[32400]],[42224.02,6197.0799999999945,0.17201238850704484,2,2,2]],[[2024,7,12,15,null,null,null,[32400]],[41190.68,5163.739999999998,0.1433299636327703,2,2,2]],[[2024,7,16,15,null,null,null,[32400]],[41275.08,5248.139999999999,0.1456726549632025,2,2,2]],[[2024,7,17,15,null,null,null,[32400]],[41097.69,5070.75,0.14074883961835224,2,2,2]],[[2024,7,18,15,null,null,null,[32400]],[40126.35,4099.409999999996,0.11378734913373148,2,2,2]],[[2024,7,19,15,null,null,null,[32400]],[40063.79,4036.8499999999985,0.11205087082055812,2,2,2]],[[2024,7,22,15,null,null,null,[32400]],[39599,3572.0599999999977,0.09914969186947316,2,2,3]],[[2024,7,23,15,null,null,null,[32400]],[39594.39,3567.449999999997,0.09902173207050048,2,2,3]],[[2024,7,24,15,null,null,null,[32400]],[39154.85,3127.909999999996,0.08682141752810525,2,2,3]],[[2024,7,25,15,null,null,null,[32400]],[37869.51,1842.5699999999997,0.05114422707007588,2,2,3]],[[2024,7,26,15,null,null,null,[32400]],[37667.41,1640.4700000000012,0.04553453609992969,2,2,3]],[[2024,7,26,15,15,2,null,[32400]],[37667.41,1640.4700000000012,0.04553453609992969,2,2,3]]]]],null,32400,37869.51,\"日経平均株価\",86400,1]]]",null,null,null,"generic"]]
57
[["di",104],["af.httprm",104,"1261915846650964379",25]]
28
[["e",4,null,null,106560]]
'''

In [10]:
def remove_before_bracket_and_after_newline(s):
    # '['前の文字を削除
    bracket_index = s.find('[')
    if bracket_index != -1:
        s = s[bracket_index:]

    # 改行以降の文字を削除
    newline_index = s.find('\n')
    if newline_index != -1:
        s = s[:newline_index]

    return s

def parse_array_string(array_string):
    def parse_helper(s):
        # 空白文字を除去
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            s = s[1:-1]
        else:
            return s.strip()  # Base case: return the stripped string itself

        result = []
        nested_level = 0
        start_index = 0

        for i, char in enumerate(s):
            if char == '[':
                if nested_level == 0:
                    start_index = i
                nested_level += 1
            elif char == ']':
                nested_level -= 1
                if nested_level == 0:
                    result.append(parse_helper(s[start_index:i+1]))
            elif char == ',' and nested_level == 0:
                result.append(parse_helper(s[start_index:i]))
                start_index = i + 1

        if start_index < len(s):
            result.append(parse_helper(s[start_index:]))

        return result

    return parse_helper(array_string)


res_text = remove_before_bracket_and_after_newline(text)

parsed_array = parse_array_string(res_text)

# print(res_text)
print(type(res_text))
print(type(parsed_array))
print(len(parsed_array[0][2][0][0][4][0][2]))
print(parsed_array[0][2][0][0][4][0][2][0])
print(parsed_array[0][2][0][0][4][0][2][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][1])
print(parsed_array[0][2][0][0][4][0][2][0][0][2])
print(parsed_array[0][2][0][0][4][0][2][0][2][0])

<class 'str'>
<class 'list'>
252
[['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['4927.93', '0', '0', '2', '2', '2'], ['4927.93', '0', '0', '2', '2', '2']]
['2024', '1', '29', '16', 'null', 'null', 'null', ['-14400'], ['-14400']]
2024
1
29
4927.93


In [ ]:
for data_array in parsed_array[0][2][0][0][4][0][2]:
    print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

2024 1 23 59.87
2024 1 23 59.87
2024 1 24 61.36
2024 1 24 61.36
2024 1 25 61.62
2024 1 25 61.62
2024 1 26 61.03
2024 1 26 61.03
2024 1 29 62.47
2024 1 29 62.47
2024 1 30 62.77
2024 1 30 62.77
2024 1 31 61.53
2024 1 31 61.53
2024 2 1 63.03
2024 2 1 63.03
2024 2 2 66.16
2024 2 2 66.16
2024 2 5 69.33
2024 2 5 69.33
2024 2 6 68.22
2024 2 6 68.22
2024 2 7 70.1
2024 2 7 70.1
2024 2 8 69.64
2024 2 8 69.64
2024 2 9 72.13
2024 2 9 72.13
2024 2 12 72.25
2024 2 12 72.25
2024 2 13 72.13
2024 2 13 72.13
2024 2 14 73.9
2024 2 14 73.9
2024 2 15 72.66
2024 2 15 72.66
2024 2 16 72.61
2024 2 16 72.61
2024 2 20 69.45
2024 2 20 69.45
2024 2 21 67.47
2024 2 21 67.47
2024 2 22 78.54
2024 2 22 78.54
2024 2 23 78.82
2024 2 23 78.82
2024 2 26 79.09
2024 2 26 79.09
2024 2 27 78.7
2024 2 27 78.7
2024 2 28 77.66
2024 2 28 77.66
2024 2 29 79.11
2024 2 29 79.11
2024 3 1 82.28
2024 3 1 82.28
2024 3 4 85.24
2024 3 4 85.24
2024 3 5 85.96
2024 3 5 85.96
2024 3 6 88.7
2024 3 6 88.7
2024 3 7 92.67
2024 3 7 92.67
2024 3 8

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import json
from datetime import datetime

import requests

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

url = "https://www.google.com/finance/?hl=ja"
driver.get(url)

time.sleep(1)

logs = driver.get_log('performance')

stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NASDAQ")
stock_code_input.send_keys(Keys.RETURN)
time.sleep(1)


for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        print(stock_code)

        stock_code_cross = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/button[2]').click()

        stock_code_reflesh = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/div/div/span/div/div/div/div[1]/input[2]')
        # stock_code_reflesh.clear()
        time.sleep(1)

        stock_code_reflesh.send_keys(stock_code)
        stock_code_reflesh.send_keys(Keys.RETURN)
        time.sleep(2)

        current_url = str(driver.current_url)
        print(current_url)
        new_url = f"{current_url}&window=6M"
        driver.get(new_url)
        logs = driver.get_log('performance')
        time.sleep(2)

        batch_request_url = None
        headers = {}
        post_data = None

        for entry in logs:
            log = json.loads(entry['message'])['message']
            try:
                if 'Network.requestWillBeSent' in log['method']:
                    request_url = log['params']['request']['url']
                    if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url and stock_code in request_url:
                        batch_request_url = request_url
                        headers = log['params']['request']['headers']
                        if 'postData' in log['params']['request']:
                            post_data = log['params']['request']['postData']
                        break
            except KeyError:
                continue

        if batch_request_url:
            print(batch_request_url)
            if post_data:
                response = requests.post(batch_request_url, headers=headers, data=post_data)
            else:
                response = requests.get(batch_request_url, headers=headers)

            print("Status Code:", response.status_code)
            print("Response Text:", response.text)
            # try:
            #     data = response.json()
            #     print(data)
            # except json.JSONDecodeError:
            #     print("レスポンスのJSONデコードに失敗しました")
        else:
            print("バッチリクエストURLが見つかりませんでした")


        def remove_before_bracket_and_after_newline(s):
            # '['前の文字を削除
            bracket_index = s.find('[')
            if bracket_index != -1:
                s = s[bracket_index:]

            # 改行以降の文字を削除
            newline_index = s.find('\n')
            if newline_index != -1:
                s = s[:newline_index]

            return s


        def parse_array_string(array_string):
            def parse_helper(s):
                # 空白文字を除去
                s = s.strip()
                if s.startswith('[') and s.endswith(']'):
                    s = s[1:-1]
                else:
                    return s.strip()  # Base case: return the stripped string itself

                result = []
                nested_level = 0
                start_index = 0

                for i, char in enumerate(s):
                    if char == '[':
                        if nested_level == 0:
                            start_index = i
                        nested_level += 1
                    elif char == ']':
                        nested_level -= 1
                        if nested_level == 0:
                            result.append(parse_helper(s[start_index:i+1]))
                    elif char == ',' and nested_level == 0:
                        result.append(parse_helper(s[start_index:i]))
                        start_index = i + 1

                if start_index < len(s):
                    result.append(parse_helper(s[start_index:]))

                return result

            return parse_helper(array_string)


        res_text = remove_before_bracket_and_after_newline(response.text)

        parsed_array = parse_array_string(res_text)

        for data_array in parsed_array[0][2][0][0][4][0][2]:
            print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

A
https://www.google.com/finance/quote/A:NYSE?hl=ja
https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=HqGpWd%2Cxh8wxf%2Co6pODe%2CnBEQBc%2CmKsvE%2CuwlMvd%2CSICF5d%2CAiCwsd%2CPr8h2e%2CyYvDpf&source-path=%2Ffinance%2Fquote%2FNDX%3AINDEXNASDAQ&f.sid=-843396336099856088&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=149257&rt=c
Status Code: 200
Response Text: )]}'

48
[["wrb.fr","yYvDpf","[]",null,null,null,"29"]]
595
[["wrb.fr","HqGpWd","[[[\"/m/02fhwv\",\"ナスダック100指数\",\"ナスダック100指数は、ナスダックに上場する、金融銘柄を除く、時価総額上位100銘柄の時価総額加重平均によって算出される株価指数である。アメリカ合衆国を代表する株価指数の一つとなっている。\",null,null,null,null,null,19032.389,19041.152,19227.2,18721.705,20690.967,14058.333,1393614291,null,null,null,null,null,null,null,null,null,null,0,0,0,1,null,\"https://ja.wikipedia.org/wiki/%E3%83%8A%E3%82%B9%E3%83%80%E3%83%83%E3%82%AF100%E6%8C%87%E6%95%B0\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,2,2,2,2,0,0,null,null,null,null,n

IndexError: list index out of range

In [4]:
import requests

BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code}:", response.status_code, response.json())
    return response.json()

days = test_get_days("NVDA")
dates = [entry['date'] for entry in days]
dates

ConnectionError: HTTPConnectionPool(host='192.168.1.222', port=8999): Max retries exceeded with url: //getdays/NVDA (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024C553E6A50>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))

In [2]:
import json
import requests

# ファイルを読み込みます
with open('stocks_values.json', 'r') as file:
    data = json.load(file)


BASE_URL = "http://192.168.1.222:8999/"

def test_get_days(stock_code):
    url = f"{BASE_URL}/getdays/{stock_code}"
    response = requests.get(url)
    print(f"GET /getdays/{stock_code}:", response.status_code, response.json())
    return response.json()

# dates = [entry['date'] for entry in days]


# JSONデータをforループで回す
for stock_symbol, entries in data.items():
    print(f"Stock Symbol: {stock_symbol}")

    days = test_get_days(stock_code)
    datas = [entry['date'] for entry in days]

    for entry in entries:
        date = entry['date']
        update_value = entry['close']
        print(f"Date: {date}, Close: {update_value}")

        if date in datas:
            data = {
                "date" : date,
                "column_name" : "value",
                "update_value" : update_value,
                "stock_code" : stock_symbol
            }

            response = requests.post(f"{BASE_URL}/update_column", json=data)
            print(response.status_code, response.text)

Stock Symbol: A
Stock Symbol: AAL
Stock Symbol: AAPL
Stock Symbol: ABBV
Stock Symbol: ABNB
Stock Symbol: ABT
Stock Symbol: ACGL
Stock Symbol: ACN
Stock Symbol: ADBE
Stock Symbol: ADI
Stock Symbol: ADM
Stock Symbol: ADP
Stock Symbol: ADSK
Stock Symbol: AEE
Stock Symbol: AEP
Stock Symbol: AES
Stock Symbol: AFL
Stock Symbol: AIG
Stock Symbol: AIZ
Stock Symbol: AJG
Stock Symbol: AKAM
Stock Symbol: ALB
Stock Symbol: ALGN
Stock Symbol: ALL
Stock Symbol: ALLE
Stock Symbol: AMAT
Stock Symbol: AMCR
Stock Symbol: AMD
Stock Symbol: AME
Stock Symbol: AMGN
Stock Symbol: AMP
Stock Symbol: AMT
Stock Symbol: AMZN
Stock Symbol: ANET
Stock Symbol: ANSS
Stock Symbol: AON
Stock Symbol: AOS
Stock Symbol: APA
Stock Symbol: APD
Stock Symbol: APH
Stock Symbol: APTV
Stock Symbol: ATO
Stock Symbol: AVGO
Stock Symbol: AVY
Stock Symbol: AWK
Stock Symbol: AXON
Stock Symbol: AXP
Stock Symbol: AZO
Stock Symbol: BA
Stock Symbol: BAC
Stock Symbol: BALL
Stock Symbol: BAX
Stock Symbol: BBWI
Stock Symbol: BBY
Stock Symbo